In [1]:
import sys
sys.path.append('../scripts')

import requests
import zipfile
import xml.etree.ElementTree as ET
import os
import pickle
from utils import download, cleanseTransWide, convColNm, cleanSalesData, cleanCmpnyNm, catOwn, catOwn2
from selenium import webdriver
import chromedriver_autoinstaller
import numpy as np
import pandas as pd

In [2]:
chromedriver_autoinstaller.install(path="../utils/")

'/Users/choidamian/Chaebol/utils/102/chromedriver'

# Download Company Code File

In [317]:
class Api:
    
    def __init__(self, apiKey):
        
        self.apiKey = apiKey
        self.corpCode = None

    def getCorpCode(self, path):

        url = "https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key=" + self.apiKey
        download(url, path)

        with zipfile.ZipFile(path, 'r') as zipObj:
            fileList = zipObj.namelist()
            target = fileList[0]
        
        loc = os.path.normpath(path)
        loc = loc.split(os.sep)
        loc = '/'.join(loc[:-1])
        
        with zipfile.ZipFile(path, 'r') as zipRef:
            zipRef.extractall(loc)

        self.corpCode = loc + '/' + target
        print(f'Corporation Code File downloaded to {self.corpCode}')

    def findCorpCode(self, corpNm):
        
        if not self.corpCode:
            print('No corporation code file found. Downloading the code file...')
            self.getCorpCode('../data/raw/opendart/corpCode/CORPCODE.zip')

        xml = ET.parse(self.corpCode)
        root = xml.getroot()

        lists = root.findall('list')

        codeList = []
        for list in lists:
            if (list.find('corp_name').text == corpNm):
                codeList.append(list.find('corp_code').text)
                
        return codeList

    def findCorpCode(self, corpNm):
        
        if not self.corpCode:
            print('No corporation code file found. Downloading the code file...')
            self.getCorpCode('../data/raw/opendart/corpCode/CORPCODE.zip')

        xml = ET.parse(self.corpCode)
        root = xml.getroot()

        lists = root.findall('list')

        codeList = []
        for list in lists:
            if (list.find('corp_name').text == corpNm):
                codeList.append(list.find('corp_code').text)
        return codeList

    def findRceptNum(self, corpNm, year):

        codeList = self.findCorpCode(corpNm)
        pblntfTy = 'j'
        pblntfDetailTy = "J004"
        pgCnt = "50"
        bgnDe = str(year) + '0501'
        endDe = str(year+1) + '0430'

        # find the firm that submitted reports to FTC whose name is <firmname>
        for corpCode in codeList:
            url = "https://opendart.fss.or.kr/api/list.json?corp_code=" + corpCode + "&crtfc_key=" + self.apiKey \
            + "&bgn_de=" + bgnDe + "&end_de=" + endDe + "&last_reprt_at=Y" + "&pblntf_ty=" + pblntfTy + \
            "&pblntf_detail_ty=" + pblntfDetailTy + "&page_count=" + pgCnt
            res = requests.get(url)
            content = res.json()
            if 'list' in content:
                break
        
        else:
            raise ValueError(f"{corpNm}: no report found for {bgnDe} ~ {endDe}")


        # find all the reports whose names contain the string '대규모기업집단현황공시[연1회공시및1/4분기용' and save as a list
        # and return rcept_no of the last report in the list
        receptNums = [report['rcept_no'] for report in content['list'] if ('대규모기업집단현황공시[연1회공시및1/4분기용(대표회사)]' \
                        in report['report_nm'])]
        
        if (corpNm in ['아모레퍼시픽그룹', '아이엠엠인베스트먼트'] and year == 2021):
            receptNums = [receptNums[0]] # select the last report received
        
        assert len(receptNums) >= 1, f'{corpNm}: no 대규모기업집단현황공시[연1회공시및1/4분기용(대표회사)] found for {bgnDe} ~ {endDe}'
        assert len(receptNums) <= 1, f'{corpNm}: Too many 대규모기업집단현황공시[연1회공시및1/4분기용(대표회사)] report found'

        return receptNums[0]

    def getSharesURL(self, corpNm, year):

        rceptNum = self.findRceptNum(corpNm, year)
        driver = webdriver.Chrome()
        driver.implicitly_wait(3)
        url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + rceptNum
        driver.get(url)

        # In the navigation menu click the link that contains the string '소유지분현황'
        driver.find_element_by_partial_link_text('소유지분현황').click()

        # get the url of the '소유지분현황' page and return it
        sharesURL = driver.find_element_by_id('ifrm').get_attribute('src')
        
        # quit
        driver.quit()

        return sharesURL

    def getSharesTable(self, corpNm, year):
        
        # get shares url
        url = self.getSharesURL(corpNm, year)
        
        # parse html tables which includes string data '동일인'(the same person in Korean) with thousands delimiter ',' 
        tables = pd.read_html(url, match='동일인', thousands=',')
        
        # convert column names
        for table in tables:
            if len(table.columns) != 12:
                print(table.head())
                
            if len(table.columns) == 11 and corpNm == '미래에셋캐피탈' and year == 2021:
                table.insert(4, '금융회사', np.nan)
            elif len(table.columns) == 10 and corpNm == '미래에셋캐피탈' and year == 2021:
                table.insert(2, '동일인과의관계', '동일인측')
                table.insert(4, '동일인과의관계3', '계열회사(국내+해외)')
            elif len(table.columns) == 9 and corpNm == '미래에셋캐피탈' and year == 2021:
                table.insert(4, '금융회사', np.nan)
                table.insert(8, '우선주주식수', np.nan)
                table.insert(9, '우선주지분율', np.nan)
                
            table.columns = ['금융회사', '소속회사명', '동일인과의관계', '동일인과의관계2', '동일인과의관계3', '성명', \
                             '보통주주식수', '보통주지분율', '우선주주식수', '우선주지분율', '합계주식수', '합계지분율']
            
        # concat tables
        df = pd.concat(tables)
        df.reset_index(inplace=True)
        df.drop(columns=['index'], inplace=True)
        df = df[df['소속회사명'] != '소속회사명']
        
        # data cleaning
        clean(df['소속회사명'])
        clean(df['성명'])
        df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']] = df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']].replace(' ', '', regex=True)
        df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']] = df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']].replace('⑧', '', regex=True)
        df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']] = df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']].replace('⑨', '', regex=True)
        df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
        df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
        df['합계주식수'] = df['합계주식수'].copy().astype(float)

        # save the data in the after_cleansing folder
        if not os.path.exists(f'../data/ownership-status/raw/{year}'):
            os.makedirs(f'../data/ownership-status/raw/{year}')
        df.to_excel(f'../data/ownership-status/raw/{year}/{corpNm}.xlsx', index=False)

        return df
    
    def getSharesTableAll(self, year):
        
        dfAppnGroupSttus = pd.read_excel(f'../data/grpSumry/appnGroupSttus/appnGroupSttus{year}.xlsx')
        dfAppnGroupSttus['repreCmpny'].replace('에스케이', 'SK', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('엘지', 'LG', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('지에스', 'GS', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('한진칼', '대한항공', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('씨제이', 'CJ', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('엘에스', 'LS', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('네이버', 'NAVER', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('에이치디씨', 'HDC', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('디엘', 'DL', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('에쓰-오일', 'S-Oil', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('오씨아이', 'OCI', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('한국투자금융지주', '한국금융지주', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('디비아이엔씨', 'DB', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('에이케이홀딩스', 'AK홀딩스', regex=True, inplace=True)
        dfAppnGroupSttus['repreCmpny'].replace('현대해상화재보험', '현대해상', regex=True, inplace=True)
        clean(dfAppnGroupSttus)
        
        with open(f'../data/utils/gNonChaebol{year}.pkl', 'rb') as f:
            gNonChaebol = pickle.load(f)
        
        for cmpny in dfAppnGroupSttus['repreCmpny']:
            if cmpny in gNonChaebol['cmpny']:
                continue
            dfShares = api.getSharesTable(cmpny, year)
            print("="* 15 + cmpny + "=" * 15)
            print(dfShares.head())
            print(dfShares.tail())
            print(dfShares['동일인과의관계'].unique())
            print(dfShares['동일인과의관계2'].unique())
            print(dfShares['동일인과의관계3'].unique())
            print("="* 15 + cmpny + "=" * 15)        

In [323]:
dfAppnGroupSttus21 = pd.read_excel('../data/grpSumry/appnGroupSttus/appnGroupSttus2021.xlsx')
dfAppnGroupSttus21['repreCmpny'].replace('에스케이', 'SK', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('엘지', 'LG', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('지에스', 'GS', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('한진칼', '대한항공', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('씨제이', 'CJ', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('엘에스', 'LS', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('네이버', 'NAVER', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('에이치디씨', 'HDC', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('디엘', 'DL', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('에쓰-오일', 'S-Oil', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('오씨아이', 'OCI', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('한국투자금융지주', '한국금융지주', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('디비아이엔씨', 'DB', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('에이케이홀딩스', 'AK홀딩스', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('현대해상화재보험', '현대해상', regex=True, inplace=True)
dfAppnGroupSttus21['repreCmpny'].replace('에이치엠엠', 'HMM', regex=True, inplace=True)
clean(dfAppnGroupSttus21)
dfAppnGroupSttus21.head(50)

,unityGrupNm,unityGrupCode,smerNm,repreCmpny,sumCmpnyCo,invstmntLmtt,entrprsCl
0,삼성,K1000032,이재용,삼성전자,59,상호출자제한집단,일반
1,현대자동차,K3000027,정의선,현대자동차,53,상호출자제한집단,일반
2,에스케이,K1000050,최태원,SK,148,상호출자제한집단,일반
3,엘지,K1000051,구광모,LG,70,상호출자제한집단,일반
4,롯데,K1000027,신동빈,롯데지주,86,상호출자제한집단,일반
5,포스코,K1000064,포스코,포스코홀딩스,33,상호출자제한집단,일반
6,한화,K1000082,김승연,한화,83,상호출자제한집단,일반
7,지에스,K3000076,허창수,GS,80,상호출자제한집단,일반
8,현대중공업,K3000052,정몽준,한국조선해양,33,상호출자제한집단,일반
9,농협,K3000091,농업협동조합중앙회,농업협동조합중앙회,58,상호출자제한집단,일반


In [324]:
gNonChaebol21 = {'grp': ['포스코', '농협', '케이티', '에쓰-오일', '대우조선해양', \
                         '케이티앤지', '대우건설', '에이치엠엠', '한국지엠', '쿠팡', '한국항공우주산업'],
                 'cmpny': ['포스코홀딩스', '농업협동조합중앙회', '케이티', 'S-Oil', '대우조선해양', \
                           '케이티앤지', '대우건설', 'HMM', '한국지엠', '쿠팡', '한국항공우주산업']}

In [320]:
with open('../data/utils/gNonChaebol21.pkl', 'wb') as f:
    pickle.dump(gNonChaebol21, f)

In [325]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
for cmpny in dfAppnGroupSttus21['repreCmpny'][47:]:
    if cmpny in gNonChaebol21['cmpny']:
        continue
    df = api.getSharesTable(cmpny, 2021)
    print("="* 15 + cmpny + "=" * 15)
    print(df.head())
    print(df.tail())
    print(df['동일인과의관계'].unique())
    print(df['동일인과의관계2'].unique())
    print(df['동일인과의관계3'].unique())
    print("="* 15 + cmpny + "=" * 15)

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
===============태광산업===============
    금융회사 소속회사명 동일인과의관계 동일인과의관계2 동일인과의관계3   성명    보통주주식수  보통주지분율 우선주주식수 우선주지분율  \
0  비금융회사  태광산업    동일인측      동일인      동일인  이호진  328189.0   29.48    NaN    NaN   
1  비금융회사  태광산업    동일인측       친족   혈족2~4촌  이원준   83370.0    7.49    NaN    NaN   
2  비금융회사  태광산업    동일인측       친족   혈족2~4촌  이동준    8918.0    0.80    NaN    NaN   
3  비금융회사  태광산업    동일인측       친족   혈족2~4촌  이태준    8982.0    0.81    NaN    NaN   
4  비금융회사  태광산업    동일인측       친족   혈족5~6촌  NaN       NaN     NaN    NaN    NaN   

      합계주식수  합계지분율  
0  328189.0  29.48  
1   83370.0   7.49  
2    8918.0   0.80  
3    8982.0   0.81  
4       NaN    NaN  
     금융회사    소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3      성명     보통주주식수  \
291  금융회사  예가람저축은행    동일인측         계열회사         계열회사  흥국생명보험   553018.0   
292  금융회사  예가람저축은행    동일인측       동일인측합계       동일인측합계     NaN  

===============삼천리===============
    금융회사 소속회사명 동일인과의관계 동일인과의관계2  동일인과의관계3           성명  보통주주식수 보통주지분율 우선주주식수  \
0  비금융회사   삼천리    동일인측      동일인       동일인          이만득  338390   8.34      -   
1  비금융회사   삼천리    동일인측       친족  배우자/혈족1촌          이은희   27045   0.67      -   
2  비금융회사   삼천리    동일인측       친족  배우자/혈족1촌          이은남   27045   0.67      -   
3  비금융회사   삼천리    동일인측       친족  배우자/혈족1촌  이은선YiEunSun   27045   0.67    NaN   
4  비금융회사   삼천리    동일인측       친족    혈족2~4촌          이은백  372070   9.18      -   

  우선주지분율   합계주식수 합계지분율  
0      -  338390  8.34  
1      -   27045  0.67  
2      -   27045  0.67  
3      -   27045  0.67  
4      -  372070  9.18  
     금융회사    소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3           성명     보통주주식수  \
634  금융회사  아네모이제2호     동일인     기타동일인관련자     기타동일인관련자            -          -   
635  금융회사  아네모이제2호     동일인       동일인측합계       동일인측합계          NaN  465851647   
636  금융회사  아네모이제2호    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주            -          -   
637  금융회사  아네모이

===============반도홀딩스===============
    금융회사  소속회사명 동일인과의관계 동일인과의관계2  동일인과의관계3   성명   보통주주식수  보통주지분율 우선주주식수  \
0  비금융회사  반도홀딩스    동일인측      동일인       동일인  권홍사  1621395  69.61%      -   
1  비금융회사  반도홀딩스    동일인측       친족  배우자/혈족1촌  권재현   700000  30.06%      -   
2  비금융회사  반도홀딩스    동일인측       친족  배우자/혈족1촌  유성애     5869   0.25%      -   
3  비금융회사  반도홀딩스    동일인측       친족  배우자/혈족1촌  권보라     1761   0.08%      -   
4  비금융회사  반도홀딩스    동일인측       친족    혈족2~4촌    -        -       -      -   

  우선주지분율    합계주식수   합계지분율  
0      -  1621395  69.61%  
1      -   700000  30.06%  
2      -     5869   0.25%  
3      -     1761   0.08%  
4      -        -       -  
      금융회사   소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3 성명 보통주주식수   보통주지분율  \
641  비금융회사  더스타아시아    동일인측       동일인측합계       동일인측합계  -  30000   75.00%   
642  비금융회사  더스타아시아    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주  -      -        -   
643  비금융회사  더스타아시아    기타주주           기타           기타  /  10000   25.00%   
644  비금융회사  더스타아시아      총계           총계      

===============엠디엠===============
    금융회사 소속회사명 동일인과의관계 동일인과의관계2  동일인과의관계3   성명  보통주주식수  보통주지분율 우선주주식수 우선주지분율  \
0  비금융회사   엠디엠    동일인측      동일인       동일인  문주현  190000  95.00%      -      -   
1  비금융회사   엠디엠    동일인측       친족  배우자/혈족1촌  민혜정   10000   5.00%      -      -   
2  비금융회사   엠디엠    동일인측       친족    혈족2~4촌    -       -       -      -      -   
3  비금융회사   엠디엠    동일인측       친족    혈족5~6촌    -       -       -      -      -   
4  비금융회사   엠디엠    동일인측       친족    인척4촌이내    -       -       -      -      -   

    합계주식수   합계지분율  
0  190000  95.00%  
1   10000   5.00%  
2       -       -  
3       -       -  
4       -       -  
     금융회사    소속회사명 동일인과의관계     동일인과의관계2     동일인과의관계3 성명    보통주주식수 보통주지분율  \
290  금융회사  한국자산캐피탈    동일인측     기타동일인관련자     기타동일인관련자  -         -      -   
291  금융회사  한국자산캐피탈    동일인측       동일인측합계       동일인측합계  /  20000000   100%   
292  금융회사  한국자산캐피탈    기타주주  동일인측이아닌최다주주  동일인측이아닌최다주주  -         -      -   
293  금융회사  한국자산캐피탈    기타주주           기타           기타  /      

# Ownership Status Data Cleansing

In [17]:
dfGroupNmId21 = pd.read_excel("../data/grpSumry/groupNmId/groupNmId2021.xlsx")
dfGroupNmId21.head()

,grpname2,grpname,grpcode
0,삼성,samsung,511
1,현대자동차,hymotor,1201801
2,에스케이,sk,535
3,엘지,lg,282
4,롯데,lotte,283


In [248]:
dfTrans = pd.read_excel("../data/transactions-mrg/2021/stacked.xlsx")
dfTrans.head()

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
0,삼성전자,삼성물산,86705.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,삼성,상장,비금융,1.660000e+14,상장,비금융,samsung,511,380725.0,810053.0
1,삼성전자,삼성바이오에피스,0.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,삼성,상장,비금융,1.660000e+14,비상장,비금융,samsung,511,380725.0,18082125.0
2,삼성전자,삼성디스플레이,196823.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,삼성,상장,비금융,1.660000e+14,비상장,비금융,samsung,511,380725.0,18082126.0
3,삼성전자,삼성전자서비스,465778.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,삼성,상장,비금융,1.660000e+14,비상장,비금융,samsung,511,380725.0,952133.0
4,삼성전자,삼성전자판매,2225711.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,삼성,상장,비금융,1.660000e+14,비상장,비금융,samsung,511,380725.0,102357.0


In [257]:
dfId1 = dfTrans[['매출회사', '매출회사id']].drop_duplicates()
dfId1.columns = ['firmname', 'firmid']
dfId1 = dfId1[~dfId1['firmid'].isna()]
dfId1.head()

,firmname,firmid
0,삼성전자,380725.0
16,삼성물산,810053.0
62,삼우종합건축사사무소,24009094.0
72,씨브이네트,136979.0
76,삼성바이오로직스,18033268.0


In [258]:
dfId2 = dfTrans[['매입회사', '매입회사id']].drop_duplicates()
dfId2.columns = ['firmname', 'firmid']
dfId2 = dfId2[~dfId2['firmid'].isna()]
dfId2.head()

,firmname,firmid
0,삼성물산,810053.0
1,삼성바이오에피스,18082125.0
2,삼성디스플레이,18082126.0
3,삼성전자서비스,952133.0
4,삼성전자판매,102357.0


In [259]:
dfId = pd.concat([dfId1, dfId2]).drop_duplicates().reset_index().drop(columns=['index'])
dfId

,firmname,firmid
0,삼성전자,3.807250e+05
1,삼성물산,8.100530e+05
2,삼우종합건축사사무소,2.400909e+07
3,씨브이네트,1.369790e+05
4,삼성바이오로직스,1.803327e+07
...,...,...
2086,그린터틀,1.302644e+07
2087,중앙리조트투자,1.802308e+09
2088,베이스에이스트링창업벤처전문사모투자,2.103801e+09
2089,베이스이스트링창업벤처전문사모투자,2.101304e+09


## 태영

In [260]:
df = pd.read_excel('../data/ownership-status/raw/2021/티와이홀딩스.xlsx')
df.head()

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율
0,비금융회사,티와이홀딩스,동일인측,동일인,동일인,윤세영,245424,0.61%,-,-,245424,0.59%
1,비금융회사,티와이홀딩스,동일인측,친족,배우자/혈족1촌,윤석민,12827810,31.93%,-,-,12827810,30.96%
2,비금융회사,티와이홀딩스,동일인측,친족,인척4촌이내,이상희,1128953,2.81%,-,-,1128953,2.73%
3,비금융회사,티와이홀딩스,동일인측,친족,친족합계,/,14202187,35.35%,-,-,14202187,34.28%
4,비금융회사,티와이홀딩스,동일인측,비영리법인,비영리법인,서암윤세영재단,2830311,7.05%,-,-,2830311,6.83%


In [261]:
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'][df['합계주식수'].str.contains('\D+', na=False, regex=True)]

257    " 해당사항 없음 "
Name: 합계주식수, dtype: object

In [262]:
# remove " 해당사항 없음 "
df['합계주식수'].replace('\D+', '', regex=True, inplace=True)

In [263]:
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [264]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
df.head()

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
0,비금융회사,티와이홀딩스,동일인측,동일인,동일인,윤세영,245424,0.61%,-,-,245424.0,0.59%,0.0,0.0
1,비금융회사,티와이홀딩스,동일인측,친족,배우자/혈족1촌,윤석민,12827810,31.93%,-,-,12827810.0,30.96%,NaN,NaN
2,비금융회사,티와이홀딩스,동일인측,친족,인척4촌이내,이상희,1128953,2.81%,-,-,1128953.0,2.73%,NaN,NaN
3,비금융회사,티와이홀딩스,동일인측,친족,친족합계,/,14202187,35.35%,-,-,14202187.0,34.28%,0.0,0.0
4,비금융회사,티와이홀딩스,동일인측,비영리법인,비영리법인,서암윤세영재단,2830311,7.05%,-,-,2830311.0,6.83%,NaN,0.0


In [265]:
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfShares.head()

,소속회사명,주식수총계
8,티와이홀딩스,41426953.0
12,군포복합개발피에프브이,5000000.0
17,디엠씨미디어,465920.0
21,리앤에스스포츠,500000.0
26,무노스,76311.0


In [266]:
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfMrg.head()

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2,주식수총계,shares
0,비금융회사,티와이홀딩스,동일인측,동일인,동일인,윤세영,245424,0.61%,-,-,245424.0,0.59%,0.0,0.0,41426953.0,0.005924
1,비금융회사,티와이홀딩스,동일인측,친족,배우자/혈족1촌,윤석민,12827810,31.93%,-,-,12827810.0,30.96%,NaN,NaN,41426953.0,0.309649
2,비금융회사,티와이홀딩스,동일인측,친족,인척4촌이내,이상희,1128953,2.81%,-,-,1128953.0,2.73%,NaN,NaN,41426953.0,0.027252
3,비금융회사,티와이홀딩스,동일인측,친족,친족합계,/,14202187,35.35%,-,-,14202187.0,34.28%,0.0,0.0,41426953.0,0.342825
4,비금융회사,티와이홀딩스,동일인측,비영리법인,비영리법인,서암윤세영재단,2830311,7.05%,-,-,2830311.0,6.83%,NaN,0.0,41426953.0,0.068321


In [267]:
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn

,소속회사명,ownCat,shares
1,걸포도시개발자산과리,99.0,0.840000
3,경산에코에너지,99.0,0.500000
5,군포복합개발피에프브이,99.0,0.646000
7,네오시티,99.0,0.310000
11,디엠씨미디어,99.0,0.085852
14,무노스,99.0,0.399955
16,문고리닷컴,99.0,0.202000
18,부산바이오에너지,99.0,0.800000
19,블루원,0.0,0.122640
23,삼계개발,99.0,0.490000


In [268]:
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfOwn2

,소속회사명,ownCat,shares
1,걸포도시개발자산과리,99.0,0.840000
3,경산에코에너지,99.0,0.500000
5,군포복합개발피에프브이,99.0,0.646000
7,네오시티,99.0,0.310000
10,디엠씨미디어,0.0,0.373626
12,디엠씨미디어,99.0,0.085852
13,리앤에스스포츠,0.0,0.300000
15,무노스,0.0,0.000026
17,무노스,99.0,0.399955
18,문고리닷컴,0.0,0.198000


In [269]:
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfMrgd

,소속회사명,ownCat,own,own2,ownername
0,걸포도시개발자산과리,99.0,0.840000,0.840000,NaN
1,경산에코에너지,99.0,0.500000,0.500000,NaN
2,군포복합개발피에프브이,99.0,0.646000,0.646000,NaN
3,네오시티,99.0,0.310000,0.310000,NaN
4,디엠씨미디어,99.0,0.085852,0.085852,NaN
5,무노스,99.0,0.399955,0.399955,NaN
6,문고리닷컴,99.0,0.202000,0.202000,NaN
7,부산바이오에너지,99.0,0.800000,0.800000,NaN
8,블루원,0.0,0.122640,0.122640,NaN
9,삼계개발,99.0,0.490000,0.490000,NaN


In [270]:
dfMrgd[dfMrgd['소속회사명']== '디엠씨미디어']

,소속회사명,ownCat,own,own2,ownername
4,디엠씨미디어,99.0,0.085852,0.085852,NaN
40,디엠씨미디어,0.0,NaN,0.373626,NaN


In [271]:
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn1

,소속회사명,ownCat,own,own2,ownername
9,군포복합개발피에프브이,1.0,0.354000,0.354000,태영건설
14,디엠씨미디어,1.0,0.540522,0.540522,에스비에스
19,리앤에스스포츠,1.0,0.700000,0.700000,SBS미디어넷
23,무노스,1.0,0.600018,0.600018,SBS미디어넷
28,문고리닷컴,1.0,0.600000,0.600000,에스비에스네오파트너스
...,...,...,...,...,...
244,창원복합행정타운개발,1.0,0.510000,0.510000,태영건설
248,천안풍세일반산업단지,1.0,0.750001,0.750001,태영건설
252,티와이신경주주택개발,1.0,1.000000,1.000000,태영건설
256,포천바이오에너지,1.0,0.510000,0.510000,태영건설


In [272]:
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']

In [273]:
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.head()

,firmname,type,own,own2,ownername
0,걸포도시개발자산과리,1.0,0.580,0.580,태영건설
1,걸포도시개발자산과리,99.0,0.840,0.840,NaN
2,경산에코에너지,1.0,0.500,0.500,태영건설
3,경산에코에너지,99.0,0.500,0.500,NaN
4,군포복합개발피에프브이,1.0,0.354,0.354,태영건설


In [274]:
dfOwn.groupby(['firmname'])['own2'].sum()

firmname
걸포도시개발자산과리     1.420000
경산에코에너지        1.000000
군포복합개발피에프브이    1.000000
네오시티           1.000000
대구남부에이엠씨       1.000000
대동산업단지         1.000000
디엠씨미디어         1.000000
리앤에스스포츠        1.000000
무노스            1.000000
문고리닷컴          1.000000
부산바이오에너지       1.400000
블루원            1.000000
블루원레저          1.000000
삼계개발           1.000000
센트로            1.000000
스마트미디어렙        1.000000
스튜디오에스         1.000000
신경주역세권공영개발     1.000000
신경주지역개발        1.754386
양주동서도로         1.000000
에스비에스          1.000000
에스비에스네오파트너스    1.000000
에스비에스디지털뉴스랩    1.000000
에스비에스미디어넷      1.000000
에스비에스미디어홀딩스    1.000000
에스비에스바이아컴      1.000000
에스비에스아이앤엠      1.000000
에스비에스에이앤티      1.000000
에스비에스엠앤씨       1.000000
에스비에스콘텐츠허브     1.000000
에스비에스플러스       1.000000
에코시스템          1.000000
에코시티           1.000000
에코시티개발         1.000000
엠시에타           1.000000
엠시에타개발         1.000000
여수엑스포환경        1.000000
여천이피에스         1.000000
유니시티           1.000000
이너시티개발         1.000000
인제스피디움         1.000000
지엠파크   

In [275]:
dfOwn[dfOwn['firmname'] == '걸포도시개발자산과리']

,firmname,type,own,own2,ownername
0,걸포도시개발자산과리,1.0,0.58,0.58,태영건설
1,걸포도시개발자산과리,99.0,0.84,0.84,NaN


In [276]:
dfOwn.iloc[0, 0] = '걸포도시개발자산관리'
dfOwn.iloc[1, 0] = '걸포도시개발자산관리'
dfOwn.iloc[1, 2] = 0.42
dfOwn.iloc[1, 3] = 0.42
dfOwn[dfOwn['firmname'] == '걸포도시개발자산관리']

,firmname,type,own,own2,ownername
0,걸포도시개발자산관리,1.0,0.58,0.58,태영건설
1,걸포도시개발자산관리,99.0,0.42,0.42,NaN


In [277]:
dfOwn[dfOwn['firmname'] == '신경주지역개발']

,firmname,type,own,own2,ownername
35,신경주지역개발,1.0,0.754386,0.754386,태영건설
36,신경주지역개발,99.0,1.000000,1.000000,NaN


In [278]:
df[df['소속회사명'] == '신경주지역개발']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
214,비금융회사,신경주지역개발,동일인측,계열회사(국내+해외),계열회사(국내+해외),태영건설,25800,43%,-,-,25800.0,43%,1.0,1.0
215,비금융회사,신경주지역개발,동일인측,동일인측합계,동일인측합계,NaN,25800,43%,-,-,25800.0,43%,NaN,NaN
216,비금융회사,신경주지역개발,기타주주,기타,기타,NaN,34200,57%,-,-,34200.0,57%,99.0,99.0
217,비금융회사,신경주지역개발,총계,총계,총계,/,60000,100%,-,-,34200.0,100%,NaN,NaN


In [279]:
dfOwn.iloc[35, 2] = 25800 / 60000
dfOwn.iloc[35, 3] = 25800 / 60000
dfOwn.iloc[36, 2] = 34200 / 60000
dfOwn.iloc[36, 3] = 34200 / 60000
dfOwn[dfOwn['firmname'] == '신경주지역개발']

,firmname,type,own,own2,ownername
35,신경주지역개발,1.0,0.43,0.43,태영건설
36,신경주지역개발,99.0,0.57,0.57,NaN


In [280]:
dfOwn[dfOwn['firmname'] == '티에스케이코퍼레이션']

,firmname,type,own,own2,ownername
99,티에스케이코퍼레이션,1.0,0.626071,0.626071,티와이홀딩스
100,티에스케이코퍼레이션,99.0,0.747857,0.747857,NaN


In [281]:
dfOwn.iloc[99, 2] = 358357 / 958357
dfOwn.iloc[99, 3] = 358357 / 958357
dfOwn[dfOwn['firmname'] == '티에스케이코퍼레이션']

,firmname,type,own,own2,ownername
99,티에스케이코퍼레이션,1.0,0.373929,0.373929,티와이홀딩스
100,티에스케이코퍼레이션,99.0,0.747857,0.747857,NaN


In [282]:
dfOwn[dfOwn['firmname'] == '티와이신경주주택개발']

,firmname,type,own,own2,ownername
102,티와이신경주주택개발,1.0,1.0,1.0,태영건설
103,티와이신경주주택개발,99.0,1.0,1.0,NaN


In [283]:
dfOwn.iloc[102, 2] = 0
dfOwn.iloc[102, 3] = 0
dfOwn[dfOwn['firmname'] == '티와이신경주주택개발']

,firmname,type,own,own2,ownername
102,티와이신경주주택개발,1.0,0.0,0.0,태영건설
103,티와이신경주주택개발,99.0,1.0,1.0,NaN


In [287]:
grpname2 = '태영'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

taeyoung


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,걸포도시개발자산관리,1.0,0.580000,0.580000,태영건설,NaN,태영,taeyoung,844
1,걸포도시개발자산관리,99.0,0.420000,0.420000,NaN,NaN,태영,taeyoung,844
2,경산에코에너지,1.0,0.500000,0.500000,태영건설,비금융회사,태영,taeyoung,844
3,경산에코에너지,99.0,0.500000,0.500000,NaN,비금융회사,태영,taeyoung,844
4,군포복합개발피에프브이,1.0,0.354000,0.354000,태영건설,비금융회사,태영,taeyoung,844
...,...,...,...,...,...,...,...,...,...
105,티와이홀딩스,99.0,0.492375,0.492375,NaN,비금융회사,태영,taeyoung,844
106,평택싸이로,1.0,1.000000,1.000000,티와이홀딩스,비금융회사,태영,taeyoung,844
107,포맷티스트,1.0,1.000000,1.000000,SBS,비금융회사,태영,taeyoung,844
108,포천바이오에너지,1.0,0.510000,0.510000,태영건설,비금융회사,태영,taeyoung,844


## 삼성

In [288]:
df = pd.read_excel('../data/ownership-status/raw/2021/삼성전자.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [289]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
미라콤아이앤씨    1.0
Name: own2, dtype: float64

In [290]:
grpname2 = '삼성'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

samsung


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,멀티캠퍼스,0.0,0.000000,0.001270,NaN,비금융회사,삼성,samsung,511
1,멀티캠퍼스,1.0,0.472432,0.472432,삼성에스디에스,비금융회사,삼성,samsung,511
2,멀티캠퍼스,1.0,0.151580,0.151580,삼성경제연구소,비금융회사,삼성,samsung,511
3,멀티캠퍼스,99.0,0.374719,0.374719,NaN,비금융회사,삼성,samsung,511
4,미라콤아이앤씨,0.0,0.000000,0.007189,NaN,비금융회사,삼성,samsung,511
...,...,...,...,...,...,...,...,...,...
212,호텔신라,1.0,0.000733,0.000733,삼성에스디아이,비금융회사,삼성,samsung,511
213,호텔신라,99.0,0.780307,0.780307,NaN,비금융회사,삼성,samsung,511
214,휴먼티에스에스,0.0,0.000000,0.000000,NaN,비금융회사,삼성,samsung,511
215,휴먼티에스에스,1.0,1.000000,1.000000,에스원,비금융회사,삼성,samsung,511


## 에스케이

In [291]:
df = pd.read_excel('../data/ownership-status/raw/2021/SK.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [292]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
에스케이바이오텍       0.000000
에스케이에코플랜트      1.000048
에스케이텔레콤        0.999903
에스케이피아이씨글로벌    1.490000
Name: own2, dtype: float64

In [293]:
df[df['소속회사명'] == '에스케이바이오텍']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
1051,비금융회사,에스케이바이오텍,동일인측,동일인,동일인,NaN,-,-,-,-,NaN,-,0.0,0.0
1052,비금융회사,에스케이바이오텍,동일인측,친족,배우자/혈족1촌,NaN,-,-,-,-,NaN,-,NaN,NaN
1053,비금융회사,에스케이바이오텍,동일인측,친족,혈족2~4촌,NaN,-,-,-,-,NaN,-,NaN,NaN
1054,비금융회사,에스케이바이오텍,동일인측,친족,혈족5~6촌,NaN,-,-,-,-,NaN,-,NaN,NaN
1055,비금융회사,에스케이바이오텍,동일인측,친족,인척4촌이내,NaN,-,-,-,-,NaN,-,NaN,NaN
1056,비금융회사,에스케이바이오텍,동일인측,친족,친족합계,/,-,-,-,-,NaN,-,0.0,0.0
1057,비금융회사,에스케이바이오텍,동일인측,비영리법인,비영리법인,NaN,-,-,-,-,NaN,-,NaN,0.0
1058,비금융회사,에스케이바이오텍,동일인측,등기된임원,등기된임원,NaN,-,-,-,-,NaN,-,NaN,0.0
1059,비금융회사,에스케이바이오텍,동일인측,자기주식,자기주식,NaN,-,-,-,-,NaN,-,NaN,0.0
1060,비금융회사,에스케이바이오텍,동일인측,계열회사(국내+해외),계열회사(국내+해외),SKPharmtecoInc.,22825332,100.00%,-,-,22825332.0,100.00%,1.0,1.0


In [294]:
dfOwn[dfOwn['firmname'] == '에스케이바이오텍']

,firmname,type,own,own2,ownername
177,에스케이바이오텍,0.0,0.0,0.0,NaN
178,에스케이바이오텍,1.0,NaN,NaN,SKPharmtecoInc.
179,에스케이바이오텍,99.0,0.0,0.0,NaN


In [295]:
dfOwn.iloc[178, 2] = 1
dfOwn.iloc[178, 3] = 1
dfOwn[dfOwn['firmname'] == '에스케이바이오텍']

,firmname,type,own,own2,ownername
177,에스케이바이오텍,0.0,0.0,0.0,NaN
178,에스케이바이오텍,1.0,1.0,1.0,SKPharmtecoInc.
179,에스케이바이오텍,99.0,0.0,0.0,NaN


In [296]:
df[df['소속회사명'] == '에스케이피아이씨글로벌']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
1577,비금융회사,에스케이피아이씨글로벌,동일인측,동일인,동일인,NaN,-,-,-,-,NaN,-,0.0,0.0
1578,비금융회사,에스케이피아이씨글로벌,동일인측,친족,배우자/혈족1촌,NaN,-,-,-,-,NaN,-,NaN,NaN
1579,비금융회사,에스케이피아이씨글로벌,동일인측,친족,혈족2~4촌,NaN,-,-,-,-,NaN,-,NaN,NaN
1580,비금융회사,에스케이피아이씨글로벌,동일인측,친족,혈족5~6촌,NaN,-,-,-,-,NaN,-,NaN,NaN
1581,비금융회사,에스케이피아이씨글로벌,동일인측,친족,인척4촌이내,NaN,-,-,-,-,NaN,-,NaN,NaN
1582,비금융회사,에스케이피아이씨글로벌,동일인측,친족,친족합계,/,-,-,-,-,NaN,-,0.0,0.0
1583,비금융회사,에스케이피아이씨글로벌,동일인측,비영리법인,비영리법인,NaN,-,-,-,-,NaN,-,NaN,0.0
1584,비금융회사,에스케이피아이씨글로벌,동일인측,등기된임원,등기된임원,NaN,-,-,-,-,NaN,-,NaN,0.0
1585,비금융회사,에스케이피아이씨글로벌,동일인측,자기주식,자기주식,NaN,-,-,-,-,NaN,-,NaN,0.0
1586,비금융회사,에스케이피아이씨글로벌,동일인측,계열회사(국내+해외),계열회사(국내+해외),에스케이씨,1020000,51.00%,-,-,1020000.0,51.00%,1.0,1.0


In [297]:
dfOwn[dfOwn['firmname'] == '에스케이피아이씨글로벌']

,firmname,type,own,own2,ownername
284,에스케이피아이씨글로벌,0.0,0.00,0.00,NaN
285,에스케이피아이씨글로벌,1.0,0.51,0.51,에스케이씨
286,에스케이피아이씨글로벌,99.0,0.98,0.98,NaN


In [298]:
dfOwn.iloc[286, 2] = 0.49
dfOwn.iloc[286, 3] = 0.49
dfOwn[dfOwn['firmname'] == '에스케이피아이씨글로벌']

,firmname,type,own,own2,ownername
284,에스케이피아이씨글로벌,0.0,0.00,0.00,NaN
285,에스케이피아이씨글로벌,1.0,0.51,0.51,에스케이씨
286,에스케이피아이씨글로벌,99.0,0.49,0.49,NaN


In [299]:
grpname2 = '에스케이'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

sk


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,강원도시가스,0.0,0.0,0.0,NaN,비금융회사,에스케이,sk,535
1,강원도시가스,1.0,1.0,1.0,에스케이이엔에스,비금융회사,에스케이,sk,535
2,강원도시가스,99.0,0.0,0.0,NaN,비금융회사,에스케이,sk,535
3,경기환경에너지,0.0,0.0,0.0,NaN,비금융회사,에스케이,sk,535
4,경기환경에너지,1.0,1.0,1.0,환경에너지,비금융회사,에스케이,sk,535
...,...,...,...,...,...,...,...,...,...
434,환경에너지,1.0,1.0,1.0,환경시설관리,비금융회사,에스케이,sk,535
435,환경에너지,99.0,0.0,0.0,NaN,비금융회사,에스케이,sk,535
436,휘찬,0.0,0.0,0.0,NaN,비금융회사,에스케이,sk,535
437,휘찬,1.0,1.0,1.0,에스케이,비금융회사,에스케이,sk,535


# 현대자동차

In [300]:
df = pd.read_excel('../data/ownership-status/raw/2021/현대자동차.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [301]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
현대위아    1.0
Name: own2, dtype: float64

In [302]:
grpname2 = '현대자동차'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

hymotor


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,그린에어,0.0,0.000000,0.000000,NaN,비금융회사,현대자동차,hymotor,1201801
1,그린에어,1.0,0.510000,0.510000,현대제철,비금융회사,현대자동차,hymotor,1201801
2,그린에어,99.0,0.490000,0.490000,NaN,비금융회사,현대자동차,hymotor,1201801
3,기아,0.0,0.017420,0.028422,NaN,비금융회사,현대자동차,hymotor,1201801
4,기아,1.0,0.338753,0.338753,현대자동차,비금융회사,현대자동차,hymotor,1201801
...,...,...,...,...,...,...,...,...,...
181,현대파텍스,0.0,0.000000,0.000000,NaN,비금융회사,현대자동차,hymotor,1201801
182,현대파텍스,1.0,0.130000,0.130000,현대모비스,비금융회사,현대자동차,hymotor,1201801
183,현대파텍스,1.0,0.560000,0.560000,현대자동차,비금융회사,현대자동차,hymotor,1201801
184,현대파텍스,1.0,0.310000,0.310000,기아,비금융회사,현대자동차,hymotor,1201801


# 엘지

In [303]:
df = pd.read_excel('../data/ownership-status/raw/2021/LG.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [304]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
고운누리      0.000000
곤지암예원     0.975000
엘지유플러스    1.000057
Name: own2, dtype: float64

In [305]:
dfOwn.to_excel("../data/ownership-input/2021/LG.xlsx", index=False)

In [306]:
df[df['소속회사명'] == '미젠스토리']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
311,비금융회사,미젠스토리,동일인측,동일인,동일인,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,0.0,0.0
312,비금융회사,미젠스토리,동일인측,친족,배우자/혈족1촌,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
313,비금융회사,미젠스토리,동일인측,친족,혈족2~4촌,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
314,비금융회사,미젠스토리,동일인측,친족,혈족5~6촌,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
315,비금융회사,미젠스토리,동일인측,친족,인척4촌이내,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
316,비금융회사,미젠스토리,동일인측,친족,친족합계,NaN,0.0,0.00%,NaN,NaN,0.0,0.00%,0.0,0.0
317,비금융회사,미젠스토리,동일인측,비영리법인,비영리법인,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,0.0
318,비금융회사,미젠스토리,동일인측,등기된임원,등기된임원,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,0.0
319,비금융회사,미젠스토리,동일인측,자기주식,자기주식,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,0.0
320,비금융회사,미젠스토리,동일인측,계열회사,계열회사,엘지생활건강,35000.0,100.00%,NaN,NaN,35000.0,100.00%,1.0,1.0


In [307]:
dfOwn[dfOwn['firmname'] == '미젠스토리']

,firmname,type,own,own2,ownername
39,미젠스토리,0.0,0.0,0.0,NaN
40,미젠스토리,1.0,1.0,1.0,엘지생활건강
41,미젠스토리,99.0,0.0,0.0,NaN


미젠스토리에 대해 공시자료에서 중복된 데이터가 발견되므로 엑셀에서 중복된 데이터를 제거함

In [308]:
df[df['소속회사명'] == '밝은누리']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
325,비금융회사,밝은누리,동일인측,동일인,동일인,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,0.0,0.0
326,비금융회사,밝은누리,동일인측,친족,배우자/혈족1촌,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
327,비금융회사,밝은누리,동일인측,친족,혈족2~4촌,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
328,비금융회사,밝은누리,동일인측,친족,혈족5~6촌,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
329,비금융회사,밝은누리,동일인측,친족,인척4촌이내,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
330,비금융회사,밝은누리,동일인측,친족,친족합계,NaN,0.0,0.00%,NaN,NaN,0.0,0.00%,0.0,0.0
331,비금융회사,밝은누리,동일인측,비영리법인,비영리법인,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,0.0
332,비금융회사,밝은누리,동일인측,등기된임원,등기된임원,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,0.0
333,비금융회사,밝은누리,동일인측,자기주식,자기주식,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,0.0
334,비금융회사,밝은누리,동일인측,계열회사,계열회사,엘지생활건강,5000.0,100.00%,NaN,NaN,5000.0,100.00%,1.0,1.0


소속회사 밝은누리에 대해 공시자료에서 중복된 데이터가 발견되므로 엑셀에서 중복된 데이터를 제거함

In [309]:
df[df['소속회사명'] == '엘지파루크']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
297,비금융회사,엘지파루크,동일인측,동일인,동일인,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,0.0,0.0
298,비금융회사,엘지파루크,동일인측,친족,배우자/혈족1촌,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
299,비금융회사,엘지파루크,동일인측,친족,혈족2~4촌,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
300,비금융회사,엘지파루크,동일인측,친족,혈족5~6촌,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
301,비금융회사,엘지파루크,동일인측,친족,인척4촌이내,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
302,비금융회사,엘지파루크,동일인측,친족,친족합계,NaN,0.0,0.00%,NaN,NaN,0.0,0.00%,0.0,0.0
303,비금융회사,엘지파루크,동일인측,비영리법인,비영리법인,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,0.0
304,비금융회사,엘지파루크,동일인측,등기된임원,등기된임원,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,0.0
305,비금융회사,엘지파루크,동일인측,자기주식,자기주식,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,0.0
306,비금융회사,엘지파루크,동일인측,계열회사,계열회사,엘지생활건강,2500.0,50.00%,NaN,NaN,2500.0,50.00%,1.0,1.0


소속회사 엘지파루크에 대해 공시자료에서 중복된 데이터가 발견되므로 엑셀에서 중복된 데이터를 제거함

In [310]:
df = pd.read_excel('../data/ownership-status/raw/2021/LG.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [311]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
고운누리      0.000000
곤지암예원     0.975000
엘지유플러스    1.000057
Name: own2, dtype: float64

In [312]:
df[df['소속회사명'] == '고운누리']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
227,비금융회사,고운누리,동일인측,동일인,동일인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
228,비금융회사,고운누리,동일인측,친족,배우자/혈족1촌,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
229,비금융회사,고운누리,동일인측,친족,혈족2~4촌,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230,비금융회사,고운누리,동일인측,친족,혈족5~6촌,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,비금융회사,고운누리,동일인측,친족,인척4촌이내,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232,비금융회사,고운누리,동일인측,친족,친족합계,NaN,0.0,0.00%,NaN,NaN,0.0,0.00%,0.0,0.0
233,비금융회사,고운누리,동일인측,비영리법인,비영리법인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
234,비금융회사,고운누리,동일인측,등기된임원,등기된임원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
235,비금융회사,고운누리,동일인측,자기주식,자기주식,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
236,비금융회사,고운누리,동일인측,계열회사,계열회사,코카콜라음료,5000.0,100.00%,NaN,NaN,5000.0,100.00%,1.0,1.0


In [313]:
df.loc[240, '합계주식수'] = 5000.0

In [314]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
곤지암예원     0.975000
엘지유플러스    1.000057
Name: own2, dtype: float64

In [315]:
# 곤지암예원의 경우 오기로 추정
df[df['소속회사명']=='곤지암예원']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
915,비금융회사,곤지암예원,동일인측,동일인,동일인,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,0.0,0.0
916,비금융회사,곤지암예원,동일인측,친족,배우자/혈족1촌,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
917,비금융회사,곤지암예원,동일인측,친족,혈족2~4촌,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
918,비금융회사,곤지암예원,동일인측,친족,혈족5~6촌,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
919,비금융회사,곤지암예원,동일인측,친족,인척4촌이내,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,NaN
920,비금융회사,곤지암예원,동일인측,친족,친족합계,NaN,0.0,0.00%,NaN,NaN,0.0,0.00%,0.0,0.0
921,비금융회사,곤지암예원,동일인측,비영리법인,비영리법인,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,0.0
922,비금융회사,곤지암예원,동일인측,등기된임원,등기된임원,석영한,120000.0,10.00%,NaN,NaN,120000.0,10.00%,NaN,0.0
923,비금융회사,곤지암예원,동일인측,자기주식,자기주식,NaN,NaN,0.00%,NaN,NaN,0.0,0.00%,NaN,0.0
924,비금융회사,곤지암예원,동일인측,계열회사,계열회사,에스앤아이코퍼레이션,1440000.0,90.00%,NaN,NaN,1440000.0,90.00%,1.0,1.0


In [316]:
grpname2 = '엘지'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

lg


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,고운누리,0.0,0.0,0.000,NaN,비금융회사,엘지,lg,282
1,고운누리,1.0,1.0,1.000,코카콜라음료,비금융회사,엘지,lg,282
2,고운누리,99.0,0.0,0.000,NaN,비금융회사,엘지,lg,282
3,곤지암예원,0.0,0.0,0.075,NaN,비금융회사,엘지,lg,282
4,곤지암예원,1.0,0.9,0.900,에스앤아이코퍼레이션,비금융회사,엘지,lg,282
...,...,...,...,...,...,...,...,...,...
206,행복마루,1.0,1.0,1.000,엘지씨엔에스,비금융회사,엘지,lg,282
207,행복마루,99.0,0.0,0.000,NaN,비금융회사,엘지,lg,282
208,헬리스타항공,0.0,0.0,0.000,NaN,비금융회사,엘지,lg,282
209,헬리스타항공,1.0,1.0,1.000,판토스,비금융회사,엘지,lg,282


# 롯데지주

In [317]:
df = pd.read_excel('../data/ownership-status/raw/2021/롯데지주.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [318]:
df.head(50)

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율
0,비금융회사,롯데지주,동일인측,동일인,동일인,신동빈,13683202,13.04%,81354,8.24%,13764556.0,13.00%
1,비금융회사,롯데지주,동일인측,친족,혈족2~4촌,신영자,3434230,3.27%,61430,6.22%,3495660.0,3.30%
2,비금융회사,롯데지주,동일인측,친족,혈족2~4촌,신동주,983029,0.94%,34962,3.54%,1017991.0,0.96%
3,비금융회사,롯데지주,동일인측,친족,혈족2~4촌,장선윤,801,0.00%,0,0.00%,801.0,0.00%
4,비금융회사,롯데지주,동일인측,친족,혈족2~4촌,장정안,77037,0.07%,0,0.00%,77037.0,0.07%
5,비금융회사,롯데지주,동일인측,친족,혈족2~4촌,신유미,42254,0.04%,0,0.00%,42254.0,0.04%
6,비금융회사,롯데지주,동일인측,친족,혈족5~6촌,조현욱,151,0.00%,11,0.00%,162.0,0.00%
7,비금융회사,롯데지주,동일인측,친족,혈족5~6촌,신영우,34,0.00%,0,0.00%,34.0,0.00%
8,비금융회사,롯데지주,동일인측,친족,혈족5~6촌,신복남,320,0.00%,0,0.00%,320.0,0.00%
9,비금융회사,롯데지주,동일인측,친족,인척4촌이내,조용완,5,0.00%,3,0.00%,8.0,0.00%


In [319]:
df['동일인과의관계2'].unique()

array(['동일인', '친족', '비영리법인', '등기된임원', '자기주식', '계열회사', '해외계열', '동일인측합계',
       '기타', '롯데지주㈜합계', '㈜호텔롯데합계', '롯데면세점제주㈜합계', '㈜부산롯데호텔합계', '롯데물산㈜합계',
       '스마일위드㈜합계', '롯데디에프리테일㈜합계', '롯데제과㈜합계', '롯데칠성음료㈜합계', '롯데푸드㈜합계',
       '동일인측이아닌최다주주', '롯데네슬레코리아㈜합계', '롯데지알에스㈜합계', '㈜롯데아사히주류합계', '㈜충북소주합계',
       '엠제이에이와인㈜합계', '씨에이치음료㈜합계', '㈜백학음료합계', '스위트위드㈜합계', '산청음료㈜합계',
       '푸드위드㈜합계', '롯데쇼핑㈜합계', '롯데역사㈜합계', '㈜코리아세븐합계', '에프알엘코리아㈜합계',
       '㈜우리홈쇼핑합계', '롯데김해개발㈜합계', '롯데수원역쇼핑타운㈜합계', '롯데지에프알㈜합계',
       '롯데송도쇼핑타운㈜합계', '한국에스티엘㈜합계', '씨에스유통㈜합계', '롯데하이마트㈜합계', '롯데인천개발㈜합계',
       '롯데인천타운㈜합계', '롯데타운동탄㈜합계', '롯데울산개발㈜합계', '롯데컬처웍스㈜합계', '롯데디엠씨개발㈜합계',
       '롯데케미칼㈜합계', '㈜케이피켐텍합계', '롯데엠시시㈜합계', '삼박엘에프티㈜합계', '데크항공㈜합계',
       '롯데미쓰이화학㈜합계', '롯데베르살리스엘라스토머스㈜합계', '롯데정밀화학㈜합계', '한덕화학㈜합계',
       '롯데지에스화학㈜합계', '한국후지필름㈜합계', '롯데알미늄㈜합계', '캐논코리아비즈니스솔루션㈜합계',
       '㈜엔젤위드합계', '롯데건설㈜합계', '롯데멤버스㈜합계', '롯데상사㈜합계', '롯데글로벌로지스㈜합계',
       '롯데부산신항로지스㈜합계', '롯데렌탈㈜합계', '제이지산업㈜합계', '롯데오토케어㈜합계', '㈜그린카합계',
       '㈜이지고합계', '서울복합물류프로젝트금융투자㈜합계',

In [320]:
df['동일인과의관계2'] = df['동일인과의관계2'].replace("^[\u3131-\u314e|\u314f-\u3163|\uac00-\ud7a3]+㈜합계$", '총계', regex=True)
df['동일인과의관계2'].unique()

array(['동일인', '친족', '비영리법인', '등기된임원', '자기주식', '계열회사', '해외계열', '동일인측합계',
       '기타', '총계', '㈜호텔롯데합계', '㈜부산롯데호텔합계', '동일인측이아닌최다주주', '㈜롯데아사히주류합계',
       '㈜충북소주합계', '㈜백학음료합계', '㈜코리아세븐합계', '㈜우리홈쇼핑합계', '㈜케이피켐텍합계',
       '㈜엔젤위드합계', '㈜그린카합계', '㈜이지고합계', '㈜대홍기획합계', '㈜모비쟆미디어합계', '㈜엠허브합계',
       '㈜롯데자이언츠합계', '㈜씨텍합계', '㈜에스디제이합계', '유한책임회사성암허심합계'], dtype=object)

In [321]:
df['동일인과의관계2'] = df['동일인과의관계2'].replace("^㈜[\u3131-\u314e|\u314f-\u3163|\uac00-\ud7a3]+합계$", '총계', regex=True)
df['동일인과의관계2'].unique()

array(['동일인', '친족', '비영리법인', '등기된임원', '자기주식', '계열회사', '해외계열', '동일인측합계',
       '기타', '총계', '동일인측이아닌최다주주', '유한책임회사성암허심합계'], dtype=object)

In [322]:
df['동일인과의관계2'] = df['동일인과의관계2'].replace("^유한책임회사[\u3131-\u314e|\u314f-\u3163|\uac00-\ud7a3]+합계$", '총계', regex=True)
df['동일인과의관계2'].unique()

array(['동일인', '친족', '비영리법인', '등기된임원', '자기주식', '계열회사', '해외계열', '동일인측합계',
       '기타', '총계', '동일인측이아닌최다주주'], dtype=object)

In [323]:
df['동일인과의관계2'] = df['동일인과의관계2'].replace("해외계열", '계열회사', regex=True)

In [324]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
롯데캐피탈    1.0
Name: own2, dtype: float64

In [325]:
grpname2 = '롯데'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

lotte


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,거제피에프브이,1.0,0.950000,0.950000,롯데자산개발,비금융회사,롯데,lotte,283
1,거제피에프브이,99.0,0.050000,0.050000,NaN,비금융회사,롯데,lotte,283
2,그린카,1.0,0.847059,0.847059,롯데렌탈,비금융회사,롯데,lotte,283
3,그린카,99.0,0.152941,0.152941,NaN,비금융회사,롯데,lotte,283
4,대홍기획,0.0,0.062366,0.112784,NaN,비금융회사,롯데,lotte,283
...,...,...,...,...,...,...,...,...,...
216,호텔롯데,1.0,0.104055,0.104055,L제9투자회사,비금융회사,롯데,lotte,283
217,호텔롯데,1.0,0.044358,0.044358,L제10투자회사,비금융회사,롯데,lotte,283
218,호텔롯데,1.0,0.033219,0.033219,L제11투자회사,비금융회사,롯데,lotte,283
219,호텔롯데,1.0,0.042013,0.042013,L제12투자회사,비금융회사,롯데,lotte,283


# 한화

In [326]:
df = pd.read_excel('../data/ownership-status/raw/2021/한화.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [327]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
아쿠아플라넷       0.00000
원오원글로벌       0.03825
한화호텔앤드리조트    1.00000
Name: own2, dtype: float64

In [328]:
df[df['소속회사명'] == '아쿠아플라넷']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
1457,비금융회사,아쿠아플라넷,동일인측,동일인,동일인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1458,비금융회사,아쿠아플라넷,동일인측,친족,배우자/혈족1촌,NaN,NaN,-,NaN,NaN,NaN,-,NaN,NaN
1459,비금융회사,아쿠아플라넷,동일인측,친족,배우자/혈족1촌,NaN,NaN,-,NaN,NaN,NaN,-,NaN,NaN
1460,비금융회사,아쿠아플라넷,동일인측,친족,배우자/혈족1촌,NaN,NaN,-,NaN,NaN,NaN,-,NaN,NaN
1461,비금융회사,아쿠아플라넷,동일인측,친족,배우자/혈족1촌,NaN,NaN,-,NaN,NaN,NaN,-,NaN,NaN
1462,비금융회사,아쿠아플라넷,동일인측,친족,혈족2~4촌,NaN,NaN,-,NaN,NaN,NaN,-,NaN,NaN
1463,비금융회사,아쿠아플라넷,동일인측,친족,혈족5~6촌,NaN,NaN,-,NaN,NaN,NaN,-,NaN,NaN
1464,비금융회사,아쿠아플라넷,동일인측,친족,인척4촌이내,NaN,NaN,-,NaN,NaN,NaN,-,NaN,NaN
1465,비금융회사,아쿠아플라넷,동일인측,친족,친족합계,NaN,-,-,-,NaN,NaN,-,0.0,0.0
1466,비금융회사,아쿠아플라넷,동일인측,비영리법인,비영리법인,NaN,NaN,-,NaN,NaN,NaN,-,NaN,0.0


In [329]:
df.loc[1479, '합계주식수'] = 100000.0

In [330]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
원오원글로벌       0.03825
한화호텔앤드리조트    1.00000
Name: own2, dtype: float64

In [331]:
df[df['소속회사명'] == '원오원글로벌']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
849,비금융회사,원오원글로벌,동일인측,동일인,동일인,NaN,NaN,-,NaN,NaN,NaN,-,0.0,0.0
850,비금융회사,원오원글로벌,동일인측,친족,배우자/혈족1촌,NaN,NaN,-,NaN,NaN,NaN,-,NaN,NaN
851,비금융회사,원오원글로벌,동일인측,친족,혈족2~4촌,NaN,NaN,-,NaN,NaN,NaN,-,NaN,NaN
852,비금융회사,원오원글로벌,동일인측,친족,혈족5~6촌,NaN,NaN,-,NaN,NaN,NaN,-,NaN,NaN
853,비금융회사,원오원글로벌,동일인측,친족,인척4촌이내,NaN,NaN,-,NaN,NaN,NaN,-,NaN,NaN
854,비금융회사,원오원글로벌,동일인측,친족,친족합계,NaN,-,-,-,NaN,NaN,-,0.0,0.0
855,비금융회사,원오원글로벌,동일인측,비영리법인,비영리법인,NaN,NaN,-,NaN,NaN,NaN,-,NaN,0.0
856,비금융회사,원오원글로벌,동일인측,등기된임원,등기된임원,NaN,NaN,-,NaN,NaN,NaN,-,NaN,0.0
857,비금융회사,원오원글로벌,동일인측,자기주식,자기주식,NaN,NaN,-,NaN,NaN,NaN,-,NaN,0.0
858,비금융회사,원오원글로벌,동일인측,계열회사,계열회사,NaN,NaN,-,NaN,NaN,NaN,-,1.0,1.0


In [332]:
grpname2 = '한화'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

hanwha


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,경기용인테크노밸리,0.0,0.00,0.00,NaN,비금융회사,한화,hanwha,913
1,경기용인테크노밸리,1.0,0.75,0.75,한화솔루션,비금융회사,한화,hanwha,913
2,경기용인테크노밸리,1.0,0.05,0.05,한화건설,비금융회사,한화,hanwha,913
3,경기용인테크노밸리,99.0,0.20,0.20,NaN,비금융회사,한화,hanwha,913
4,김해테크노밸리,0.0,0.00,0.00,NaN,비금융회사,한화,hanwha,913
...,...,...,...,...,...,...,...,...,...
272,해사랑태양광,1.0,1.00,1.00,한화글로벌에셋,비금융회사,한화,hanwha,913
273,해사랑태양광,99.0,0.00,0.00,NaN,비금융회사,한화,hanwha,913
274,해오름태양광발전소,0.0,0.00,0.00,NaN,비금융회사,한화,hanwha,913
275,해오름태양광발전소,1.0,1.00,1.00,한화솔루션,비금융회사,한화,hanwha,913


# 지에스

In [333]:
df = pd.read_excel('../data/ownership-status/raw/2021/GS.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [334]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
파르나스호텔    0.324444
Name: own2, dtype: float64

In [335]:
df

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
0,비금융회사,지에스,동일인측,동일인,동일인,허창수,4417695,4.75%,NaN,0.00%,4417695.0,4.66%,0.0,0.0
1,비금융회사,지에스,동일인측,친족,배우자/혈족1촌,HuhAliceYoonyoung,323917,0.35%,NaN,0.00%,323917.0,0.34%,NaN,NaN
2,비금융회사,지에스,동일인측,친족,배우자/혈족1촌,허윤홍,494888,0.53%,NaN,0.00%,494888.0,0.52%,NaN,NaN
3,비금융회사,지에스,동일인측,친족,혈족2~4촌,허광수,2033983,2.19%,NaN,0.00%,2033983.0,2.15%,NaN,NaN
4,비금융회사,지에스,동일인측,친족,혈족2~4촌,허남각,1956670,2.11%,NaN,0.00%,1956670.0,2.07%,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,비금융회사,프로케어,동일인측,동일인측합계,동일인측합계,NaN,100000,100.00%,NaN,0.00%,100000.0,100.00%,NaN,NaN
492,비금융회사,프로케어,총계,NaN,NaN,NaN,100000,100.00%,NaN,0.00%,100000.0,100.00%,NaN,NaN
493,금융회사,지베스코자산운용,동일인측,계열회사(국내+해외),계열회사(국내+해외),지베스코,920000,100.00%,NaN,0.00%,920000.0,100.00%,1.0,1.0
494,금융회사,지베스코자산운용,동일인측,동일인측합계,동일인측합계,NaN,920000,100.00%,NaN,0.00%,920000.0,100.00%,NaN,NaN


In [336]:
dfOwn

,firmname,type,own,own2,ownername
0,가승개발,1.0,0.45,0.45,승산
1,가승개발,99.0,0.55,0.55,NaN
2,강남메트로,1.0,0.39,0.39,지에스건설
3,강남메트로,1.0,0.04,0.04,지에스네오텍
4,강남메트로,99.0,0.57,0.57,NaN
...,...,...,...,...,...
130,향산2도시개발,1.0,0.50,0.50,지에스건설
131,향산2도시개발,99.0,0.50,0.50,NaN
132,후레쉬미트,1.0,0.51,0.51,지에스리테일
133,후레쉬미트,99.0,0.49,0.49,NaN


In [337]:
grpname2 = '지에스'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

gs


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,가승개발,1.0,0.45,0.45,승산,비금융회사,지에스,gs,2703408
1,가승개발,99.0,0.55,0.55,NaN,비금융회사,지에스,gs,2703408
2,강남메트로,1.0,0.39,0.39,지에스건설,비금융회사,지에스,gs,2703408
3,강남메트로,1.0,0.04,0.04,지에스네오텍,비금융회사,지에스,gs,2703408
4,강남메트로,99.0,0.57,0.57,NaN,비금융회사,지에스,gs,2703408
...,...,...,...,...,...,...,...,...,...
130,향산2도시개발,1.0,0.50,0.50,지에스건설,비금융회사,지에스,gs,2703408
131,향산2도시개발,99.0,0.50,0.50,NaN,비금융회사,지에스,gs,2703408
132,후레쉬미트,1.0,0.51,0.51,지에스리테일,비금융회사,지에스,gs,2703408
133,후레쉬미트,99.0,0.49,0.49,NaN,비금융회사,지에스,gs,2703408


# 현대중공업

In [338]:
df = pd.read_excel('../data/ownership-status/raw/2021/한국조선해양.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [339]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
태백풍력발전           1.000000
현대일렉트릭앤에너지시스템    1.000009
Name: own2, dtype: float64

In [340]:
grpname2 = '현대중공업'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

hhi


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,아비커스,0.0,0.00,0.00,NaN,비금융회사,현대중공업,hhi,1901305
1,아비커스,1.0,1.00,1.00,현대중공업지주,비금융회사,현대중공업,hhi,1901305
2,아비커스,99.0,0.00,0.00,NaN,비금융회사,현대중공업,hhi,1901305
3,아산카카오메디컬데이터,0.0,0.00,0.05,NaN,비금융회사,현대중공업,hhi,1901305
4,아산카카오메디컬데이터,1.0,0.45,0.45,현대미래파트너스,비금융회사,현대중공업,hhi,1901305
...,...,...,...,...,...,...,...,...,...
94,현대코스모,1.0,0.50,0.50,현대오일뱅크,비금융회사,현대중공업,hhi,1901305
95,현대코스모,99.0,0.50,0.50,NaN,비금융회사,현대중공업,hhi,1901305
96,현대코어모션,0.0,0.00,0.00,NaN,비금융회사,현대중공업,hhi,1901305
97,현대코어모션,1.0,1.00,1.00,현대건설기계,비금융회사,현대중공업,hhi,1901305


# 신세계

In [341]:
df = pd.read_excel('../data/ownership-status/raw/2021/신세계.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [342]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
신세계       0.0
인천신세계    10.0
Name: own2, dtype: float64

In [343]:
df[df['소속회사명'] == '신세계']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
0,비금융회사,신세계,동일인측,동일인,동일인,이명희,984518,10.00,-,-,984518.0,10.00,0.0,0.0
1,비금융회사,신세계,동일인측,친족,배우자/혈족1촌,정유경,1827521,18.56,-,-,1827521.0,18.56,NaN,NaN
2,비금융회사,신세계,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
3,비금융회사,신세계,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
4,비금융회사,신세계,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
5,비금융회사,신세계,동일인측,친족,친족합계,/,1827521,18.56,-,-,1827521.0,18.56,0.0,0.0
6,비금융회사,신세계,동일인측,비영리법인,비영리법인,-,-,-,-,-,NaN,-,NaN,0.0
7,비금융회사,신세계,동일인측,등기된임원,등기된임원,차정호,560,0.00,-,-,560.0,-,NaN,0.0
8,비금융회사,신세계,동일인측,등기된임원,등기된임원,박주형,345,0.00,-,-,345.0,-,NaN,0.0
9,비금융회사,신세계,동일인측,자기주식,자기주식,신세계,8774,0.09,-,-,8774.0,NaN,NaN,0.0


In [344]:
df.loc[15, '합계주식수'] = 9845181

In [345]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
신세계       0.286609
인천신세계    10.000000
Name: own2, dtype: float64

In [346]:
df[df['소속회사명'] == '인천신세계']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
46,비금융회사,인천신세계,동일인측,동일인,동일인,이명희,-,-,-,-,NaN,-,0.0,0.0
47,비금융회사,인천신세계,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
48,비금융회사,인천신세계,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
49,비금융회사,인천신세계,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
50,비금융회사,인천신세계,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
51,비금융회사,인천신세계,동일인측,친족,친족합계,/,-,-,-,-,NaN,-,0.0,0.0
52,비금융회사,인천신세계,동일인측,비영리법인,비영리법인,-,-,-,-,-,NaN,-,NaN,0.0
53,비금융회사,인천신세계,동일인측,등기된임원,등기된임원,-,-,-,-,-,NaN,-,NaN,0.0
54,비금융회사,인천신세계,동일인측,자기주식,자기주식,-,-,-,-,-,NaN,-,NaN,0.0
55,비금융회사,인천신세계,동일인측,계열회사(국내+해외),계열회사(국내+해외),신세계,1494000,90.00,-,-,1494000.0,90.00,1.0,1.0


In [347]:
df.loc[60, '합계주식수'] = 1660000

In [348]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
신세계    0.286609
Name: own2, dtype: float64

In [349]:
grpname2 = '신세계'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

shinsegae


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,광주신세계,0.0,0.520831,0.526182,NaN,비금융회사,신세계,shinsegae,577
1,광주신세계,1.0,0.104169,0.104169,신세계,비금융회사,신세계,shinsegae,577
2,광주신세계,99.0,0.369649,0.369649,NaN,비금융회사,신세계,shinsegae,577
3,까사미아,0.0,0.000000,0.000000,NaN,비금융회사,신세계,shinsegae,577
4,까사미아,1.0,0.956798,0.956798,신세계,비금융회사,신세계,shinsegae,577
...,...,...,...,...,...,...,...,...,...
139,제주소주,1.0,1.000000,1.000000,이마트,비금융회사,신세계,shinsegae,577
140,제주소주,99.0,0.000000,0.000000,NaN,비금융회사,신세계,shinsegae,577
141,조선호텔앤리조트,0.0,0.000000,0.000000,NaN,비금융회사,신세계,shinsegae,577
142,조선호텔앤리조트,1.0,0.999592,0.999592,이마트,비금융회사,신세계,shinsegae,577


# 씨제이

In [350]:
df = pd.read_excel('../data/ownership-status/raw/2021/CJ.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [351]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]


firmname
씨제이프레시웨이    0.890495
프레시원광주      1.000000
Name: own2, dtype: float64

In [352]:
df[df['소속회사명'] == '씨제이프레시웨이']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
462,비금융회사,씨제이프레시웨이,동일인측,동일인,동일인,이재현,70000,0.59,NaN,0.0,70000.0,0.59,0.0,0.0
463,비금융회사,씨제이프레시웨이,동일인측,친족,배우자/혈족1촌,-,NaN,0.00,NaN,0.0,NaN,0.00,NaN,NaN
464,비금융회사,씨제이프레시웨이,동일인측,친족,배우자/혈족1촌,-,NaN,0.00,NaN,0.0,NaN,0.00,NaN,NaN
465,비금융회사,씨제이프레시웨이,동일인측,친족,혈족2~4촌,-,NaN,0.00,NaN,0.0,NaN,0.00,NaN,NaN
466,비금융회사,씨제이프레시웨이,동일인측,친족,혈족2~4촌,-,NaN,0.00,NaN,0.0,NaN,0.00,NaN,NaN
467,비금융회사,씨제이프레시웨이,동일인측,친족,혈족5~6촌,-,NaN,0.00,NaN,0.0,NaN,0.00,NaN,NaN
468,비금융회사,씨제이프레시웨이,동일인측,친족,혈족5~6촌,-,NaN,0.00,NaN,0.0,NaN,0.00,NaN,NaN
469,비금융회사,씨제이프레시웨이,동일인측,친족,인척4촌이내,-,NaN,0.00,NaN,0.0,NaN,0.00,NaN,NaN
470,비금융회사,씨제이프레시웨이,동일인측,친족,인척4촌이내,-,NaN,0.00,NaN,0.0,NaN,0.00,NaN,NaN
471,비금융회사,씨제이프레시웨이,동일인측,친족,친족합계,-,-,0.00,-,0.0,NaN,0.00,0.0,0.0


In [353]:
grpname2 = '씨제이'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

cj


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,농업회사법인제이팜스,0.0,0.000000,0.000000,NaN,비금융회사,씨제이,cj,720
1,농업회사법인제이팜스,1.0,0.700005,0.700005,씨제이프레시웨이,비금융회사,씨제이,cj,720
2,농업회사법인제이팜스,99.0,0.299995,0.299995,NaN,비금융회사,씨제이,cj,720
3,다다엠앤씨,0.0,0.000000,0.000000,NaN,비금융회사,씨제이,cj,720
4,다다엠앤씨,1.0,1.000000,1.000000,씨제이이엔엠,비금융회사,씨제이,cj,720
...,...,...,...,...,...,...,...,...,...
233,형제푸드,1.0,1.000000,1.000000,씨제이프레시웨이,비금융회사,씨제이,cj,720
234,형제푸드,99.0,0.000000,0.000000,NaN,비금융회사,씨제이,cj,720
235,화앤담픽쳐스,0.0,0.000000,0.000000,NaN,비금융회사,씨제이,cj,720
236,화앤담픽쳐스,1.0,1.000000,1.000000,스튜디오드래곤,비금융회사,씨제이,cj,720


# 한진

In [354]:
df = pd.read_excel('../data/ownership-status/raw/2021/대한항공.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [355]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
대한항공        1.000088
세계혼재항공화물    2.000000
Name: own2, dtype: float64

In [356]:
df[df['소속회사명'] == '세계혼재항공화물'] 

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
367,비금융회사,세계혼재항공화물,동일인측,친족,혈족2~4촌,이상영,30000,30%,-,-,30000.0,30%,NaN,NaN
368,비금융회사,세계혼재항공화물,동일인측,친족,인척4촌이내,손희종,30000,30%,-,-,30000.0,30%,NaN,NaN
369,비금융회사,세계혼재항공화물,동일인측,친족,친족합계,/,60000,60%,-,-,60000.0,60%,0.0,0.0
370,비금융회사,세계혼재항공화물,동일인측,등기된임원,등기된임원,김태익,30000,30%,-,-,30000.0,30%,NaN,0.0
371,비금융회사,세계혼재항공화물,동일인측,등기된임원,등기된임원,조영우,10000,10%,-,-,10000.0,10%,NaN,0.0
372,비금융회사,세계혼재항공화물,동일인측,등기된임원,등기된임원,/,100000,100%,-,-,100000.0,100%,NaN,0.0
373,비금융회사,세계혼재항공화물,총계,총계,총계,/,100000,100%,-,-,100000.0,100%,NaN,NaN


In [357]:
df.loc[372, '합계주식수'] = 0

In [358]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
대한항공    1.000088
Name: own2, dtype: float64

In [359]:
grpname2 = '한진'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

hanjin


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,대한항공,0.0,0.000382,0.009670,NaN,비금융회사,한진,hanjin,926
1,대한항공,1.0,0.275747,0.275747,한진칼,비금융회사,한진,hanjin,926
2,대한항공,99.0,0.714670,0.714670,NaN,비금융회사,한진,hanjin,926
3,더블유에이씨항공서비스,1.0,1.000000,1.000000,세계혼재항공화물,비금융회사,한진,hanjin,926
4,부산글로벌물류센터,1.0,0.510000,0.510000,한진,비금융회사,한진,hanjin,926
...,...,...,...,...,...,...,...,...,...
72,한진칼,0.0,0.223352,0.253404,NaN,비금융회사,한진,hanjin,926
74,한진칼,99.0,0.746596,0.746596,NaN,비금융회사,한진,hanjin,926
75,항공종합서비스,0.0,0.000000,0.000000,NaN,비금융회사,한진,hanjin,926
76,항공종합서비스,1.0,1.000000,1.000000,대한항공,비금융회사,한진,hanjin,926


# 두산

In [360]:
df = pd.read_excel('../data/ownership-status/raw/2021/두산.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [361]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [362]:
dfOwn

,firmname,type,own,own2,ownername
0,네오트랜스,0.0,0.000000,0.000000,NaN
1,네오트랜스,1.0,0.428600,0.428600,두산건설
2,네오트랜스,99.0,0.571400,0.571400,NaN
3,두산,0.0,0.369954,0.615105,NaN
4,두산,1.0,NaN,NaN,-
...,...,...,...,...,...
71,이큐브솔루션,1.0,0.600000,0.600000,두산인프라코어
72,이큐브솔루션,99.0,0.400000,0.400000,NaN
73,한컴,0.0,0.000000,0.000000,NaN
74,한컴,1.0,1.000000,1.000000,오리콤


In [363]:
grpname2 = '두산'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

doosan


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,네오트랜스,0.0,0.000000,0.000000,NaN,비금융회사,두산,doosan,391
1,네오트랜스,1.0,0.428600,0.428600,두산건설,비금융회사,두산,doosan,391
2,네오트랜스,99.0,0.571400,0.571400,NaN,비금융회사,두산,doosan,391
3,두산,0.0,0.369954,0.615105,NaN,비금융회사,두산,doosan,391
6,두산,99.0,0.384895,0.384895,NaN,비금융회사,두산,doosan,391
...,...,...,...,...,...,...,...,...,...
71,이큐브솔루션,1.0,0.600000,0.600000,두산인프라코어,비금융회사,두산,doosan,391
72,이큐브솔루션,99.0,0.400000,0.400000,NaN,비금융회사,두산,doosan,391
73,한컴,0.0,0.000000,0.000000,NaN,비금융회사,두산,doosan,391
74,한컴,1.0,1.000000,1.000000,오리콤,비금융회사,두산,doosan,391


# 엘에스

In [364]:
df = pd.read_excel('../data/ownership-status/raw/2021/LS.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [365]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
지엘마린    0.0
Name: own2, dtype: float64

In [366]:
df[df['소속회사명'] == '지엘마린']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
340,비금융회사,지엘마린,동일인측,동일인,동일인,-,-,-,-,-,NaN,_,0.0,0.0
341,비금융회사,지엘마린,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,_,NaN,NaN
342,비금융회사,지엘마린,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,_,NaN,NaN
343,비금융회사,지엘마린,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,_,NaN,NaN
344,비금융회사,지엘마린,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,_,NaN,NaN
345,비금융회사,지엘마린,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,_,NaN,NaN
346,비금융회사,지엘마린,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,_,NaN,NaN
347,비금융회사,지엘마린,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,_,NaN,NaN
348,비금융회사,지엘마린,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,_,NaN,NaN
349,비금융회사,지엘마린,동일인측,친족,친족합계,-,-,-,-,-,NaN,_,0.0,0.0


In [367]:
df.loc[358, '합계주식수'] = 33202000

In [368]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [369]:
grpname2 = '엘에스'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

ls


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,E1,0.0,0.415449,0.580587,NaN,비금융회사,엘에스,ls,2402204
2,E1,99.0,0.419413,0.419413,NaN,비금융회사,엘에스,ls,2402204
3,E1물류,0.0,0.000000,0.000000,NaN,비금융회사,엘에스,ls,2402204
4,E1물류,1.0,1.000000,1.000000,E1,비금융회사,엘에스,ls,2402204
5,E1물류,99.0,0.000000,0.000000,NaN,비금융회사,엘에스,ls,2402204
...,...,...,...,...,...,...,...,...,...
192,한성피씨건설,1.0,0.216667,0.216667,예스코홀딩스,비금융회사,엘에스,ls,2402204
194,한성피씨건설,99.0,0.000000,0.000000,NaN,비금융회사,엘에스,ls,2402204
195,흥업,0.0,0.000000,0.000000,NaN,비금융회사,엘에스,ls,2402204
196,흥업,1.0,1.000000,1.000000,LS네트웍스,비금융회사,엘에스,ls,2402204


# 부영

In [370]:
df = pd.read_excel('../data/ownership-status/raw/2021/부영.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [371]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [372]:
dfOwn

,firmname,type,own,own2,ownername
0,광영토건,0.0,0.511611,1.000000,NaN
1,광영토건,1.0,NaN,NaN,-
2,광영토건,99.0,0.000000,0.000000,NaN
3,남광건설산업,0.0,1.000000,1.000000,NaN
4,남광건설산업,1.0,NaN,NaN,-
...,...,...,...,...,...
64,천원종합개발,1.0,0.995495,0.995495,부영주택
65,천원종합개발,99.0,0.004505,0.004505,NaN
66,한라일보사,0.0,0.489999,0.489999,NaN
67,한라일보사,1.0,NaN,NaN,-


In [373]:
grpname2 = '부영'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

booyoung


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,광영토건,0.0,0.511611,1.000000,NaN,비금융회사,부영,booyoung,16032
2,광영토건,99.0,0.000000,0.000000,NaN,비금융회사,부영,booyoung,16032
3,남광건설산업,0.0,1.000000,1.000000,NaN,비금융회사,부영,booyoung,16032
5,남광건설산업,99.0,0.000000,0.000000,NaN,비금융회사,부영,booyoung,16032
6,남양개발,0.0,1.000000,1.000000,NaN,비금융회사,부영,booyoung,16032
8,남양개발,99.0,0.000000,0.000000,NaN,비금융회사,부영,booyoung,16032
9,대화도시가스,0.0,0.950000,1.000000,NaN,비금융회사,부영,booyoung,16032
11,대화도시가스,99.0,0.000000,0.000000,NaN,비금융회사,부영,booyoung,16032
12,더클래식씨씨,0.0,0.000000,0.000000,NaN,비금융회사,부영,booyoung,16032
13,더클래식씨씨,1.0,0.988517,0.988517,부영주택,비금융회사,부영,booyoung,16032


# 카카오

In [374]:
df = pd.read_excel('../data/ownership-status/raw/2021/카카오.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [375]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
손노리          1.000000
스타쉽엔터테인먼트    1.000000
카카오          1.153696
Name: own2, dtype: float64

In [376]:
df[df['소속회사명'] == '카카오']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
0,비금융회사,카카오,동일인측,동일인,동일인,김범수,59124547,13.32,-,-,59124547.0,13.32,0.0,0.0
1,비금융회사,카카오,동일인측,친족,배우자/혈족1촌,형미선,282000,0.06,-,-,282000.0,0.06,NaN,NaN
2,비금융회사,카카오,동일인측,친족,배우자/혈족1촌,김상빈,270000,0.06,-,-,270000.0,0.06,NaN,NaN
3,비금융회사,카카오,동일인측,친족,배우자/혈족1촌,김예빈,268000,0.06,-,-,268000.0,0.06,NaN,NaN
4,비금융회사,카카오,동일인측,친족,혈족2~4촌,김명희,90000,0.02,-,-,90000.0,0.02,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,비금융회사,카카오,동일인측,기타동일인관련자,기타동일인관련자,-,-,-,-,-,NaN,-,NaN,NaN
72,비금융회사,카카오,동일인측,동일인측합계,동일인측합계,/,120172504,27.08,-,-,61047957.0,27.08,NaN,NaN
73,비금융회사,카카오,기타주주,동일인측이아닌최다주주,동일인측이아닌최다주주,-,-,-,-,-,NaN,-,99.0,99.0
74,비금융회사,카카오,기타주주,기타,기타,/,323636801,72.92,-,-,323636801.0,72.92,99.0,99.0


In [377]:
df.loc[75, '합계주식수'] = 443809305

In [378]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
손노리          1.0
스타쉽엔터테인먼트    1.0
Name: own2, dtype: float64

In [379]:
grpname2 = '카카오'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

kakao


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,가승개발,0.0,0.000000,0.000000,NaN,비금융회사,카카오,kakao,36027
1,가승개발,1.0,0.550000,0.550000,카카오브이엑스,비금융회사,카카오,kakao,36027
2,가승개발,99.0,0.450000,0.450000,NaN,비금융회사,카카오,kakao,36027
3,골프와친구,0.0,0.000000,1.000000,NaN,비금융회사,카카오,kakao,36027
5,골프와친구,99.0,0.000000,0.000000,NaN,비금융회사,카카오,kakao,36027
...,...,...,...,...,...,...,...,...,...
360,하이라인엔터테인먼트,1.0,0.770000,0.770000,스타쉽엔터테인먼트,비금융회사,카카오,kakao,36027
361,하이라인엔터테인먼트,99.0,0.000000,0.000000,NaN,비금융회사,카카오,kakao,36027
362,하이업엔터테인먼트,0.0,0.000000,0.300000,NaN,비금융회사,카카오,kakao,36027
363,하이업엔터테인먼트,1.0,0.399999,0.399999,카카오엔터테인먼트,비금융회사,카카오,kakao,36027


# DL

In [380]:
df = pd.read_excel('../data/ownership-status/raw/2021/DL.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [381]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
디엘건설    1.000573
Name: own2, dtype: float64

In [382]:
grpname2 = 'DL'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

DL


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,글래드호텔앤리조트,1.0,1.000000,1.000000,디엘,비금융회사,DL,DL,321
1,김해동서터널,1.0,0.390000,0.390000,디엘건설,비금융회사,DL,DL,321
2,김해동서터널,1.0,0.370000,0.370000,디엘이앤씨,비금융회사,DL,DL,321
3,김해동서터널,99.0,0.240000,0.240000,NaN,비금융회사,DL,DL,321
4,대림,0.0,0.527770,0.673421,NaN,비금융회사,DL,DL,321
...,...,...,...,...,...,...,...,...,...
64,험프리에스엘큐원,1.0,0.353003,0.353003,디엘,비금융회사,DL,DL,321
65,험프리에스엘큐원,1.0,0.099999,0.099999,대림,비금융회사,DL,DL,321
66,험프리에스엘큐원,99.0,0.546997,0.546997,NaN,비금융회사,DL,DL,321
67,효제피에프브이,1.0,0.660000,0.660000,디엘이앤씨,비금융회사,DL,DL,321


# 미래에셋

In [383]:
df = pd.read_excel('../data/ownership-status/raw/2021/미래에셋캐피탈.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [384]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
미래에셋생명보험            1.000002
미래에셋증권글로벌제이호사모투자    0.000000
서울공항리무진             1.000000
수원학교사랑              0.850000
신성장동력그린퓨처사모투자       0.000000
Name: own2, dtype: float64

In [385]:
df[df['소속회사명'] == '수원학교사랑']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
370,비금융회사,수원학교사랑,동일인측,계열회사(국내+해외),NaN,미래에셋생명보험,NaN,NaN,45014493,100%,45014493.0,100%,1.0,1.0
371,비금융회사,수원학교사랑,동일인측,동일인측합계,NaN,/,NaN,NaN,45014493,100%,45014493.0,100%,NaN,NaN
372,비금융회사,수원학교사랑,기타주주,기타,NaN,덕동종합건설,1323956,17%,NaN,NaN,NaN,NaN,99.0,99.0
373,비금융회사,수원학교사랑,기타주주,기타,NaN,에이치디에스자산관리,2647911,34%,NaN,NaN,NaN,NaN,99.0,99.0
374,비금융회사,수원학교사랑,기타주주,기타,NaN,영동건설,1323956,17%,NaN,NaN,NaN,NaN,99.0,99.0
375,비금융회사,수원학교사랑,기타주주,기타,NaN,한동건설,2647911,34%,NaN,NaN,NaN,NaN,99.0,99.0
376,비금융회사,수원학교사랑,총계,총계,NaN,/,7943734,100%,45014493,100%,52958227.0,100%,NaN,NaN


In [386]:
df.loc[372, '합계주식수'] = 1323956
df.loc[373, '합계주식수'] = 2647911
df.loc[374, '합계주식수'] = 1323956
df.loc[375, '합계주식수'] = 2647911

In [387]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
미래에셋생명보험            1.000002
미래에셋증권글로벌제이호사모투자    0.000000
서울공항리무진             1.000000
신성장동력그린퓨처사모투자       0.000000
Name: own2, dtype: float64

In [388]:
df[df['소속회사명'] == '신성장동력그린퓨처사모투자']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
308,금융회사,신성장동력그린퓨처사모투자,동일인측,계열회사(국내+해외),NaN,미래에셋증권,-,-,-,-,NaN,-,1.0,1.0
309,금융회사,신성장동력그린퓨처사모투자,동일인측,동일인측합계,NaN,/,-,-,-,-,NaN,-,NaN,NaN
310,금융회사,신성장동력그린퓨처사모투자,기타주주,동일인측이아닌최다주주,NaN,한국산업은행,-,-,-,-,NaN,-,99.0,99.0
311,금융회사,신성장동력그린퓨처사모투자,기타주주,기타,NaN,화이텍인베스트먼트외,-,-,-,-,NaN,-,99.0,99.0
312,금융회사,신성장동력그린퓨처사모투자,총계,총계,NaN,/,-,-,-,-,NaN,-,NaN,NaN


In [389]:
grpname2 = '미래에셋'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

miraeasset


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,그로쓰파트너,1.0,1.000000,1.000000,미래에셋증권코리아제일호사모투자,금융회사,미래에셋,miraeasset,2602105
1,멀티에셋자산운용,0.0,0.000000,0.000000,NaN,금융회사,미래에셋,miraeasset,2602105
2,멀티에셋자산운용,1.0,1.000000,1.000000,미래에셋자산운용,금융회사,미래에셋,miraeasset,2602105
3,멀티에셋자산운용,99.0,0.000000,0.000000,NaN,금융회사,미래에셋,miraeasset,2602105
4,미래에셋글로벌유니콘사모투자,1.0,0.867857,0.867857,미래에셋증권,금융회사,미래에셋,miraeasset,2602105
...,...,...,...,...,...,...,...,...,...
90,와이케이디벨롭먼트,1.0,0.500000,0.500000,미래에셋컨설팅,비금융회사,미래에셋,miraeasset,2602105
91,와이케이디벨롭먼트,99.0,0.500000,0.500000,NaN,비금융회사,미래에셋,miraeasset,2602105
92,자스민제일호,1.0,1.000000,1.000000,미래에셋파트너스제구호사모투자,금융회사,미래에셋,miraeasset,2602105
93,코에프씨미래에셋그로쓰챔프2010의4호사모투자,1.0,0.030029,0.030029,미래에셋자산운용,금융회사,미래에셋,miraeasset,2602105


# 현대백화점

In [390]:
df = pd.read_excel('../data/ownership-status/raw/2021/현대백화점.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [391]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
한섬라이프앤    1.0
Name: own2, dtype: float64

In [392]:
grpname2 = '현대백화점'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

ehyundai


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,씨엔에스푸드시스템,0.0,0.000000,0.000000,NaN,비금융회사,현대백화점,ehyundai,38035
1,씨엔에스푸드시스템,1.0,1.000000,1.000000,현대그린푸드,비금융회사,현대백화점,ehyundai,38035
3,씨엔에스푸드시스템,99.0,0.000000,0.000000,NaN,비금융회사,현대백화점,ehyundai,38035
4,에버다임락툴,0.0,0.000000,0.000000,NaN,비금융회사,현대백화점,ehyundai,38035
5,에버다임락툴,1.0,1.000000,1.000000,현대에버다임,비금융회사,현대백화점,ehyundai,38035
...,...,...,...,...,...,...,...,...,...
98,현대퓨처넷,99.0,0.313741,0.313741,NaN,비금융회사,현대백화점,ehyundai,38035
99,현대홈쇼핑,0.0,0.000000,0.046021,NaN,비금융회사,현대백화점,ehyundai,38035
100,현대홈쇼핑,1.0,0.250125,0.250125,현대그린푸드,비금융회사,현대백화점,ehyundai,38035
101,현대홈쇼핑,1.0,0.158042,0.158042,현대백화점,비금융회사,현대백화점,ehyundai,38035


# 금호아시아나

In [393]:
df = pd.read_excel('../data/ownership-status/raw/2021/금호건설.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [394]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [395]:
dfOwn

,firmname,type,own,own2,ownername
0,금호건설,0.0,0.003352,0.032614,NaN
1,금호건설,1.0,0.442107,0.442107,금호고속
2,금호건설,99.0,0.525279,0.525279,NaN
3,금호고속,0.0,0.688887,0.917037,NaN
4,금호고속,1.0,0.020741,0.020741,케이에이
...,...,...,...,...,...
75,케이에프,99.0,0.000000,0.000000,NaN
76,케이오,0.0,0.000000,1.000000,NaN
77,케이오,99.0,0.000000,0.000000,NaN
78,케이지,0.0,0.000000,1.000000,NaN


In [396]:
grpname2 = '금호아시아나'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

kumho


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,금호건설,0.0,0.003352,0.032614,NaN,비금융회사,금호아시아나,kumho,183
1,금호건설,1.0,0.442107,0.442107,금호고속,비금융회사,금호아시아나,kumho,183
2,금호건설,99.0,0.525279,0.525279,NaN,비금융회사,금호아시아나,kumho,183
3,금호고속,0.0,0.688887,0.917037,NaN,비금융회사,금호아시아나,kumho,183
4,금호고속,1.0,0.020741,0.020741,케이에이,비금융회사,금호아시아나,kumho,183
...,...,...,...,...,...,...,...,...,...
75,케이에프,99.0,0.000000,0.000000,NaN,비금융회사,금호아시아나,kumho,183
76,케이오,0.0,0.000000,1.000000,NaN,비금융회사,금호아시아나,kumho,183
77,케이오,99.0,0.000000,0.000000,NaN,비금융회사,금호아시아나,kumho,183
78,케이지,0.0,0.000000,1.000000,NaN,비금융회사,금호아시아나,kumho,183


# 셀트리온

In [397]:
df = pd.read_excel('../data/ownership-status/raw/2021/셀트리온홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [398]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
셀트리온홀딩스    1.0
Name: own2, dtype: float64

In [399]:
dfOwn

,firmname,type,own,own2,ownername
0,셀트리온,0.0,0.001431,0.014254,NaN
1,셀트리온,1.0,0.200107,0.200107,셀트리온홀딩스
2,셀트리온,1.0,0.021158,0.021158,셀트리온스킨큐어
3,셀트리온,99.0,0.764481,0.764481,NaN
4,셀트리온스킨큐어,0.0,0.742452,0.828185,NaN
5,셀트리온스킨큐어,99.0,0.171815,0.171815,NaN
6,셀트리온엔터테인먼트,1.0,1.000000,1.000000,셀트리온홀딩스
7,셀트리온엔터테인먼트,99.0,0.000000,0.000000,NaN
8,셀트리온제약,0.0,0.000149,0.002757,NaN
9,셀트리온제약,1.0,0.549327,0.549327,셀트리온


In [400]:
grpname2 = '셀트리온'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

celltrion


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,셀트리온,0.0,0.001431,0.014254,NaN,비금융회사,셀트리온,celltrion,1803209
1,셀트리온,1.0,0.200107,0.200107,셀트리온홀딩스,비금융회사,셀트리온,celltrion,1803209
2,셀트리온,1.0,0.021158,0.021158,셀트리온스킨큐어,비금융회사,셀트리온,celltrion,1803209
3,셀트리온,99.0,0.764481,0.764481,NaN,비금융회사,셀트리온,celltrion,1803209
4,셀트리온스킨큐어,0.0,0.742452,0.828185,NaN,비금융회사,셀트리온,celltrion,1803209
5,셀트리온스킨큐어,99.0,0.171815,0.171815,NaN,비금융회사,셀트리온,celltrion,1803209
6,셀트리온엔터테인먼트,1.0,1.000000,1.000000,셀트리온홀딩스,비금융회사,셀트리온,celltrion,1803209
7,셀트리온엔터테인먼트,99.0,0.000000,0.000000,NaN,비금융회사,셀트리온,celltrion,1803209
8,셀트리온제약,0.0,0.000149,0.002757,NaN,비금융회사,셀트리온,celltrion,1803209
9,셀트리온제약,1.0,0.549327,0.549327,셀트리온,비금융회사,셀트리온,celltrion,1803209


# 한국투자금융

In [401]:
df = pd.read_excel('../data/ownership-status/raw/2021/한국금융지주.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [402]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
이큐파트너스제1호환경서비스사모투자            1.000000
케이엔반도체투자                      1.000000
한국투자글로벌제약산업육성사모투자             1.000000
한국투자온기업그린모빌리티제1호기업재무안정사모투자    0.898182
Name: own2, dtype: float64

In [403]:
df[df['소속회사명'] == '한국투자온기업그린모빌리티제1호기업재무안정사모투자']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
306,금융회사,한국투자온기업그린모빌리티제1호기업재무안정사모투자,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
307,금융회사,한국투자온기업그린모빌리티제1호기업재무안정사모투자,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
308,금융회사,한국투자온기업그린모빌리티제1호기업재무안정사모투자,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
309,금융회사,한국투자온기업그린모빌리티제1호기업재무안정사모투자,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
310,금융회사,한국투자온기업그린모빌리티제1호기업재무안정사모투자,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
311,금융회사,한국투자온기업그린모빌리티제1호기업재무안정사모투자,동일인측,친족,친족합계,/,-,-,-,-,NaN,-,0.0,0.0
312,금융회사,한국투자온기업그린모빌리티제1호기업재무안정사모투자,동일인측,비영리법인,비영리법인,-,-,-,-,-,NaN,-,NaN,0.0
313,금융회사,한국투자온기업그린모빌리티제1호기업재무안정사모투자,동일인측,등기된임원,등기된임원,-,-,-,-,-,NaN,-,NaN,0.0
314,금융회사,한국투자온기업그린모빌리티제1호기업재무안정사모투자,동일인측,자기주식,자기주식,-,-,-,-,-,NaN,-,NaN,0.0
315,금융회사,한국투자온기업그린모빌리티제1호기업재무안정사모투자,동일인측,계열회사(국내+해외),계열회사(국내+해외),한국투자증권,10000000000,18.18,-,-,1.000000e+10,18.18,1.0,1.0


In [404]:
df.loc[316, '동일인과의관계2'] = '계열회사(국내+해외)'

In [405]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
이큐파트너스제1호환경서비스사모투자    1.0
케이엔반도체투자              1.0
한국투자글로벌제약산업육성사모투자     1.0
Name: own2, dtype: float64

In [406]:
grpname2 = '한국투자금융'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

kinv


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,그린환경기술,0.0,0.000000,0.000000,NaN,비금융회사,한국투자금융,kinv,2702504
1,그린환경기술,1.0,1.000000,1.000000,이큐그린제이차,비금융회사,한국투자금융,kinv,2702504
2,그린환경기술,99.0,0.000000,0.000000,NaN,비금융회사,한국투자금융,kinv,2702504
3,도시환경,0.0,0.000000,0.000000,NaN,비금융회사,한국투자금융,kinv,2702504
4,도시환경,1.0,0.882000,0.882000,이큐그린제이차,비금융회사,한국투자금융,kinv,2702504
...,...,...,...,...,...,...,...,...,...
87,한투에스지기업재무안정사모투자,1.0,0.117417,0.117417,한국투자프라이빗에쿼티,금융회사,한국투자금융,kinv,2702504
88,한투에스지기업재무안정사모투자,99.0,0.765166,0.765166,NaN,금융회사,한국투자금융,kinv,2702504
89,한투에스지제일호,0.0,0.000000,0.000000,NaN,금융회사,한국투자금융,kinv,2702504
90,한투에스지제일호,1.0,1.000000,1.000000,한투에스지기업재무안정사모투자,금융회사,한국투자금융,kinv,2702504


# 교보생명보험

In [407]:
df = pd.read_excel('../data/ownership-status/raw/2021/교보생명보험.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [408]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [409]:
dfOwn

,firmname,type,own,own2,ownername
0,교보라이프플래닛생명보험,0.0,0.000000,0.000000,NaN
1,교보라이프플래닛생명보험,1.0,1.000000,1.000000,교보생명보험
2,교보라이프플래닛생명보험,99.0,0.000000,0.000000,NaN
3,교보리얼코,0.0,0.000000,0.000000,NaN
4,교보리얼코,1.0,1.000000,1.000000,교보생명보험
5,교보리얼코,99.0,0.000000,0.000000,NaN
6,교보문고,0.0,0.000000,0.000000,NaN
7,교보문고,1.0,1.000000,1.000000,교보생명보험
8,교보문고,99.0,0.000000,0.000000,NaN
9,교보생명보험,0.0,0.369047,0.369060,NaN


In [410]:
grpname2 = '교보생명보험'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

kyobodirect


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,교보라이프플래닛생명보험,0.0,0.000000,0.000000,NaN,금융회사,교보생명보험,kyobodirect,145
1,교보라이프플래닛생명보험,1.0,1.000000,1.000000,교보생명보험,금융회사,교보생명보험,kyobodirect,145
2,교보라이프플래닛생명보험,99.0,0.000000,0.000000,NaN,금융회사,교보생명보험,kyobodirect,145
3,교보리얼코,0.0,0.000000,0.000000,NaN,비금융회사,교보생명보험,kyobodirect,145
4,교보리얼코,1.0,1.000000,1.000000,교보생명보험,비금융회사,교보생명보험,kyobodirect,145
5,교보리얼코,99.0,0.000000,0.000000,NaN,비금융회사,교보생명보험,kyobodirect,145
6,교보문고,0.0,0.000000,0.000000,NaN,비금융회사,교보생명보험,kyobodirect,145
7,교보문고,1.0,1.000000,1.000000,교보생명보험,비금융회사,교보생명보험,kyobodirect,145
8,교보문고,99.0,0.000000,0.000000,NaN,비금융회사,교보생명보험,kyobodirect,145
9,교보생명보험,0.0,0.369047,0.369060,NaN,금융회사,교보생명보험,kyobodirect,145


# 네이버

In [411]:
df = pd.read_excel('../data/ownership-status/raw/2021/NAVER.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [412]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
네이버    0.999999
Name: own2, dtype: float64

In [413]:
grpname2 = '네이버'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

naver


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,그레이해쉬,0.0,0.000000,0.090693,NaN,비금융회사,네이버,naver,1502101
1,그레이해쉬,1.0,0.769853,0.769853,라인플러스,비금융회사,네이버,naver,1502101
2,그레이해쉬,99.0,0.139454,0.139454,NaN,비금융회사,네이버,naver,1502101
3,그린웹서비스,0.0,0.000000,0.000000,NaN,비금융회사,네이버,naver,1502101
4,그린웹서비스,1.0,1.000000,1.000000,네이버아이앤에스,비금융회사,네이버,naver,1502101
...,...,...,...,...,...,...,...,...,...
142,플레이리스트,1.0,0.290208,0.290208,네이버웹툰,비금융회사,네이버,naver,1502101
143,플레이리스트,99.0,0.419584,0.419584,NaN,비금융회사,네이버,naver,1502101
144,피그,0.0,0.000000,0.000000,NaN,비금융회사,네이버,naver,1502101
145,피그,1.0,1.000000,1.000000,라인게임즈,비금융회사,네이버,naver,1502101


# HDC

In [414]:
df = pd.read_excel('../data/ownership-status/raw/2021/HDC.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [415]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
에이치디씨현대산업개발    1.0
Name: own2, dtype: float64

In [416]:
dfOwn

,firmname,type,own,own2,ownername
0,광명문화복합단지자산관리,0.0,0.000000,0.000000,NaN
1,광명문화복합단지자산관리,1.0,0.600000,0.600000,에이치디씨현대산업개발
2,광명문화복합단지자산관리,99.0,0.400000,0.400000,NaN
3,미래비아이,0.0,0.000000,0.000000,NaN
4,미래비아이,1.0,0.644494,0.644494,부동산일일사
...,...,...,...,...,...
82,통영에코파워,1.0,1.000000,1.000000,에이치디씨
83,통영에코파워,99.0,0.000000,0.000000,NaN
84,호텔에이치디씨,0.0,0.000000,0.000000,NaN
85,호텔에이치디씨,1.0,1.000000,1.000000,에이치디씨현대산업개발


In [417]:
grpname2 = '에이치디씨'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

HDC


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,광명문화복합단지자산관리,0.0,0.000000,0.000000,NaN,비금융회사,에이치디씨,HDC,1101907
1,광명문화복합단지자산관리,1.0,0.600000,0.600000,에이치디씨현대산업개발,비금융회사,에이치디씨,HDC,1101907
2,광명문화복합단지자산관리,99.0,0.400000,0.400000,NaN,비금융회사,에이치디씨,HDC,1101907
3,미래비아이,0.0,0.000000,0.000000,NaN,비금융회사,에이치디씨,HDC,1101907
4,미래비아이,1.0,0.644494,0.644494,부동산일일사,비금융회사,에이치디씨,HDC,1101907
...,...,...,...,...,...,...,...,...,...
82,통영에코파워,1.0,1.000000,1.000000,에이치디씨,비금융회사,에이치디씨,HDC,1101907
83,통영에코파워,99.0,0.000000,0.000000,NaN,비금융회사,에이치디씨,HDC,1101907
84,호텔에이치디씨,0.0,0.000000,0.000000,NaN,비금융회사,에이치디씨,HDC,1101907
85,호텔에이치디씨,1.0,1.000000,1.000000,에이치디씨현대산업개발,비금융회사,에이치디씨,HDC,1101907


# 효성

In [418]:
df = pd.read_excel('../data/ownership-status/raw/2021/효성.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [419]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
효성티앤에스    1.0
Name: own2, dtype: float64

In [420]:
grpname2 = '효성'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

hyosung


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,갤럭시아디바이스,0.0,1.000000,1.000000,NaN,비금융회사,효성,hyosung,971
2,갤럭시아디바이스,99.0,0.000000,0.000000,NaN,비금융회사,효성,hyosung,971
3,갤럭시아머니트리,0.0,0.338904,0.373031,NaN,비금융회사,효성,hyosung,971
4,갤럭시아머니트리,1.0,0.166758,0.166758,효성아이티엑스,비금융회사,효성,hyosung,971
5,갤럭시아머니트리,1.0,0.084105,0.084105,트리니티에셋매지니먼트,비금융회사,효성,hyosung,971
...,...,...,...,...,...,...,...,...,...
149,효성프리미어모터스,1.0,1.000000,1.000000,신동진,비금융회사,효성,hyosung,971
150,효성프리미어모터스,99.0,0.000000,0.000000,NaN,비금융회사,효성,hyosung,971
151,효성화학,0.0,0.235854,0.239822,NaN,비금융회사,효성,hyosung,971
152,효성화학,1.0,0.201710,0.201710,효성,비금융회사,효성,hyosung,971


# 영풍

In [421]:
df = pd.read_excel('../data/ownership-status/raw/2021/영풍.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [422]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [423]:
dfOwn

,firmname,type,own,own2,ownername
0,고려아연,0.0,0.053914,0.184120,NaN
1,고려아연,1.0,0.274870,0.274870,영풍
2,고려아연,1.0,0.015639,0.015639,영풍정밀
3,고려아연,1.0,0.009889,0.009889,유미개발
4,고려아연,1.0,0.001125,0.001125,테라닉스
...,...,...,...,...,...
93,클린코리아,99.0,0.000000,0.000000,NaN
94,테라닉스,0.0,0.005357,0.029196,NaN
95,테라닉스,1.0,0.500935,0.500935,코리아써키트
96,테라닉스,1.0,0.416238,0.416238,영풍


In [424]:
grpname2 = '영풍'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

youngpoong


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,고려아연,0.0,0.053914,0.184120,NaN,비금융회사,영풍,youngpoong,632
1,고려아연,1.0,0.274870,0.274870,영풍,비금융회사,영풍,youngpoong,632
2,고려아연,1.0,0.015639,0.015639,영풍정밀,비금융회사,영풍,youngpoong,632
3,고려아연,1.0,0.009889,0.009889,유미개발,비금융회사,영풍,youngpoong,632
4,고려아연,1.0,0.001125,0.001125,테라닉스,비금융회사,영풍,youngpoong,632
...,...,...,...,...,...,...,...,...,...
93,클린코리아,99.0,0.000000,0.000000,NaN,비금융회사,영풍,youngpoong,632
94,테라닉스,0.0,0.005357,0.029196,NaN,비금융회사,영풍,youngpoong,632
95,테라닉스,1.0,0.500935,0.500935,코리아써키트,비금융회사,영풍,youngpoong,632
96,테라닉스,1.0,0.416238,0.416238,영풍,비금융회사,영풍,youngpoong,632


# 하림

In [425]:
df = pd.read_excel('../data/ownership-status/raw/2021/하림지주.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [426]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [427]:
dfOwn

,firmname,type,own,own2,ownername
0,경우,0.0,0.8,0.8,NaN
1,경우,1.0,0.2,0.2,올품
2,경우,1.0,NaN,NaN,-
3,경우,99.0,0.0,0.0,NaN
4,글라이드,0.0,0.0,0.0,NaN
...,...,...,...,...,...
217,한국썸벧,99.0,0.0,0.0,NaN
218,한국인베스트먼트,0.0,0.0,0.0,NaN
219,한국인베스트먼트,1.0,1.0,1.0,올품
220,한국인베스트먼트,1.0,NaN,NaN,-


In [428]:
dfOwn.firmname.unique()

array(['경우', '글라이드', '농업회사법인늘푸른', '농업회사법인봉산지피', '농업회사법인봉화', '농업회사법인비앤피',
       '농업회사법인선진한마을', '농업회사법인순우리한우', '농업회사법인에이치비씨', '농업회사법인익산',
       '농업회사법인주원산오리', '농업회사법인팜스코바이오인티', '농업회사법인팜엔코', '농업회사법인피그앤그린',
       '농업회사법인하이포크스마트팜', '동림', '디디에프엔비', '맥시칸', '선진', '선진브릿지랩', '선진에프에스',
       '선진팜', '선진햄', '세티', '싱그린에프에스', '애그리로보텍', '에버미라클', '에이치에스푸드',
       '에코캐피탈', '엑셀로이콰인알앤디센터', '엔디', '엔바이콘', '엔에스쇼핑', '엔에스홈쇼핑미디어센터', '올품',
       '제이에이치제이', '제일사료', '지포레', '참트레이딩', '청운농업회사법인', '팜스코', '팜스코푸드',
       '팬오션', '포스에스엠', '하림', '하림산업', '하림애니멀클리닉', '하림엠에스', '하림유통', '하림지주',
       '하림펫푸드', '하림푸드', '한강씨엠', '한국썸벧', '한국인베스트먼트'], dtype=object)

In [429]:
grpname2 = '하림'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

harim


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,경우,0.0,0.8,0.8,NaN,비금융회사,하림,harim,24062
1,경우,1.0,0.2,0.2,올품,비금융회사,하림,harim,24062
3,경우,99.0,0.0,0.0,NaN,비금융회사,하림,harim,24062
4,글라이드,0.0,0.0,0.0,NaN,비금융회사,하림,harim,24062
5,글라이드,1.0,1.0,1.0,엔에스쇼핑,비금융회사,하림,harim,24062
...,...,...,...,...,...,...,...,...,...
215,한국썸벧,1.0,1.0,1.0,제일사료,비금융회사,하림,harim,24062
217,한국썸벧,99.0,0.0,0.0,NaN,비금융회사,하림,harim,24062
218,한국인베스트먼트,0.0,0.0,0.0,NaN,비금융회사,하림,harim,24062
219,한국인베스트먼트,1.0,1.0,1.0,올품,비금융회사,하림,harim,24062


# 케이씨씨

In [430]:
df = pd.read_excel('../data/ownership-status/raw/2021/케이씨씨.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [431]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
-       0.0
금강레저    1.0
Name: own2, dtype: float64

In [432]:
df = df[~(df['소속회사명'] == '-')]

In [433]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
금강레저    1.0
Name: own2, dtype: float64

In [434]:
grpname2 = '케이씨씨'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

kcc


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,금강레저,0.0,0.765000,0.765000,NaN,비금융회사,케이씨씨,kcc,184
1,금강레저,1.0,0.205000,0.205000,케이씨씨,비금융회사,케이씨씨,kcc,184
2,금강레저,1.0,0.030000,0.030000,케이씨씨건설,비금융회사,케이씨씨,kcc,184
3,금강레저,99.0,0.000000,0.000000,NaN,비금융회사,케이씨씨,kcc,184
4,대호포장,0.0,1.000000,1.000000,NaN,비금융회사,케이씨씨,kcc,184
6,대호포장,99.0,0.000000,0.000000,NaN,비금융회사,케이씨씨,kcc,184
7,동주,0.0,0.000000,0.000000,NaN,비금융회사,케이씨씨,kcc,184
8,동주,1.0,1.000000,1.000000,세우실업,비금융회사,케이씨씨,kcc,184
9,동주,99.0,0.000000,0.000000,NaN,비금융회사,케이씨씨,kcc,184
10,동주상사,0.0,1.000000,1.000000,NaN,비금융회사,케이씨씨,kcc,184


# 엔엑스씨

In [435]:
df = pd.read_excel('../data/ownership-status/raw/2021/엔엑스씨.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [436]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
엔엑스씨    1.674918
Name: own2, dtype: float64

In [437]:
df[df['소속회사명'] == '엔엑스씨']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
0,비금융회사,엔엑스씨,동일인측,동일인,동일인,김정주,1963000,67.49%,-,-,1963000.0,67.49%,0.0,0.0
1,비금융회사,엔엑스씨,동일인측,친족,배우자/혈족1촌,유정현,856000,29.43%,-,-,856000.0,29.43%,NaN,NaN
2,비금융회사,엔엑스씨,동일인측,친족,배우자/혈족1촌,김정민,19750,0.68%,-,-,19750.0,0.68%,NaN,NaN
3,비금융회사,엔엑스씨,동일인측,친족,배우자/혈족1촌,김정윤,19750,0.68%,-,-,19750.0,0.68%,NaN,NaN
4,비금융회사,엔엑스씨,동일인측,친족,친족합계,/,2858500,98.28%,-,-,2858500.0,98.28%,0.0,0.0
5,비금융회사,엔엑스씨,동일인측,계열회사(국내+해외),계열회사(국내+해외),유한책임회사와이즈키즈,50000,1.72%,-,-,50000.0,1.72%,1.0,1.0
6,비금융회사,엔엑스씨,동일인측,동일인측합계,동일인측합계,/,2908500,100.00%,-,-,2908500.0,100.00%,NaN,NaN
7,비금융회사,엔엑스씨,총계,총계,총계,/,2908500,100.00%,-,-,2908500.0,100.00%,NaN,NaN


In [438]:
df.loc[4, '합계주식수'] = 2858500.0 - 1963000.0

In [439]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [442]:
grpname2 = '넥슨'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

nexon


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,네오플,1.0,1.000000,1.000000,넥슨코리아,비금융회사,넥슨,nexon,1902205
1,넥슨네트웍스,1.0,1.000000,1.000000,넥슨코리아,비금융회사,넥슨,nexon,1902205
2,넥슨스페이스,1.0,1.000000,1.000000,넥슨코리아,비금융회사,넥슨,nexon,1902205
3,넥슨지티,0.0,NaN,0.029885,NaN,비금융회사,넥슨,nexon,1902205
4,넥슨지티,1.0,0.631556,0.631556,넥슨코리아,비금융회사,넥슨,nexon,1902205
5,넥슨지티,99.0,0.338560,0.338560,NaN,비금융회사,넥슨,nexon,1902205
6,넥슨커뮤니케이션즈,1.0,1.000000,1.000000,넥슨코리아,비금융회사,넥슨,nexon,1902205
7,넥슨코리아,1.0,1.000000,1.000000,"NEXONCo.,Ltd.",비금융회사,넥슨,nexon,1902205
8,니트로스튜디오,1.0,0.500000,0.500000,넥슨코리아,비금융회사,넥슨,nexon,1902205
9,니트로스튜디오,99.0,0.500000,0.500000,NaN,비금융회사,넥슨,nexon,1902205


# 넷마블

In [443]:
df = pd.read_excel('../data/ownership-status/raw/2021/넷마블.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [444]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
넷마블엔투    1.0
잼팟       1.0
Name: own2, dtype: float64

In [445]:
df[df['소속회사명'] == '넷마블에프앤씨']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
153,비금융회사,넷마블에프앤씨,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
154,비금융회사,넷마블에프앤씨,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
155,비금융회사,넷마블에프앤씨,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
156,비금융회사,넷마블에프앤씨,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
157,비금융회사,넷마블에프앤씨,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
158,비금융회사,넷마블에프앤씨,동일인측,친족,친족합계,-,-,-,-,-,NaN,-,0.0,0.0
159,비금융회사,넷마블에프앤씨,동일인측,비영리법인,비영리법인,-,-,-,-,-,NaN,-,NaN,0.0
160,비금융회사,넷마블에프앤씨,동일인측,등기된임원,등기된임원,서우원,252335,2.04,-,-,252335.0,2.04,NaN,0.0
161,비금융회사,넷마블에프앤씨,동일인측,등기된임원,등기된임원,정철호,102755,0.83,NaN,NaN,102755.0,0.83,NaN,0.0
162,비금융회사,넷마블에프앤씨,동일인측,자기주식,자기주식,넷마블에프앤씨,263005,2.12,-,-,263005.0,2.12,NaN,0.0


In [446]:
def catOwn(row):
    if row['동일인과의관계2'] == '동일인':
        return 0
    elif row['동일인과의관계2'] == '친족' and row['동일인과의관계3'] == '친족합계':
        return 0
    elif row['동일인과의관계2'] in ['계열회사(국내+해외)', '계열회사', '계열회사(국내)', '해외계열']:
        return 1
    elif row['동일인과의관계'] in ['기타주주', '기타']:
        return 99

def catOwn2(row):
    if row['동일인과의관계2'] in ['동일인', '비영리법인', '등기된임원', '자기주식']:
        return 0
    elif row['동일인과의관계2'] == '친족' and row['동일인과의관계3'] == '친족합계':
        return 0
    elif row['동일인과의관계2'] in ['계열회사(국내+해외)', '계열회사', '계열회사(국내)', '해외계열']:
        return 1
    elif row['동일인과의관계'] in ['기타주주', '기타']:
        return 99

In [447]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
넷마블엔투    1.0
잼팟       1.0
Name: own2, dtype: float64

In [448]:
grpname2 = '넷마블'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

netmarble


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,구로발게임즈,0.0,0.006393,0.006393,NaN,비금융회사,넷마블,netmarble,360280
1,구로발게임즈,1.0,0.993607,0.993607,넷마블,비금융회사,넷마블,netmarble,360280
2,구로발게임즈,99.0,0.000000,0.000000,NaN,비금융회사,넷마블,netmarble,360280
3,넷마블,0.0,0.241171,0.288984,NaN,비금융회사,넷마블,netmarble,360280
5,넷마블,99.0,0.711016,0.711016,NaN,비금융회사,넷마블,netmarble,360280
...,...,...,...,...,...,...,...,...,...
68,키링,1.0,0.755007,0.755007,넷마블엔투,비금융회사,넷마블,netmarble,360280
69,키링,99.0,0.052498,0.052498,NaN,비금융회사,넷마블,netmarble,360280
70,포천맑은물,0.0,0.000000,0.000000,NaN,비금융회사,넷마블,netmarble,360280
71,포천맑은물,1.0,0.699838,0.699838,코웨이엔텍,비금융회사,넷마블,netmarble,360280


# 호반건설

In [449]:
df = pd.read_excel('../data/ownership-status/raw/2021/호반건설.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [450]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
마륵파크     1.0
호반서서울    1.0
Name: own2, dtype: float64

In [451]:
grpname2 = '호반건설'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

hoban


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,광주방송,0.0,NaN,0.100000,NaN,비금융회사,호반건설,hoban,26023
1,광주방송,1.0,0.165875,0.165875,호반건설,비금융회사,호반건설,hoban,26023
2,광주방송,1.0,0.130000,0.130000,호반프라퍼티,비금융회사,호반건설,hoban,26023
3,광주방송,99.0,0.604125,0.604125,NaN,비금융회사,호반건설,hoban,26023
4,그린산업,1.0,0.510000,0.510000,호반호텔앤리조트,비금융회사,호반건설,hoban,26023
...,...,...,...,...,...,...,...,...,...
67,호반티비엠,1.0,1.000000,1.000000,호반산업,비금융회사,호반건설,hoban,26023
68,호반프라퍼티,0.0,0.516129,1.000000,NaN,비금융회사,호반건설,hoban,26023
69,호반호텔앤리조트,1.0,1.000000,1.000000,호반건설,비금융회사,호반건설,hoban,26023
70,화랑관사비티엘,1.0,0.692401,0.692401,호반산업,비금융회사,호반건설,hoban,26023


# SM

In [452]:
df = pd.read_excel('../data/ownership-status/raw/2021/티케이케미칼.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [453]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
삼환기업      1.0
에스엠중공업    1.0
Name: own2, dtype: float64

In [454]:
grpname2 = 'SM'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

SM


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,경남기업,0.0,0.000000,4.947709e-03,NaN,비금융회사,SM,SM,192
1,경남기업,1.0,0.659143,6.591429e-01,동아건설산업,비금융회사,SM,SM,192
2,경남기업,1.0,0.282503,2.825031e-01,대한상선,비금융회사,SM,SM,192
3,경남기업,99.0,0.053406,5.340627e-02,NaN,비금융회사,SM,SM,192
4,남선알미늄,0.0,0.000000,1.900676e-07,NaN,비금융회사,SM,SM,192
...,...,...,...,...,...,...,...,...,...
176,한덕철광산업,0.0,0.000000,0.000000e+00,NaN,비금융회사,SM,SM,192
177,한덕철광산업,1.0,1.000000,1.000000e+00,대한상선,비금융회사,SM,SM,192
178,한덕철광산업,99.0,0.000000,0.000000e+00,NaN,비금융회사,SM,SM,192
179,한통엔지니어링,0.0,0.998377,9.983774e-01,NaN,비금융회사,SM,SM,192


# DB

In [455]:
df = pd.read_excel('../data/ownership-status/raw/2021/DB.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [456]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
DB금융투자    1.0
Name: own2, dtype: float64

In [457]:
grpname2 = 'DB'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

DB


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,DBInc.,0.0,0.394799,0.488721,NaN,비금융회사,DB,DB,365
2,DBInc.,99.0,0.511279,0.511279,NaN,비금융회사,DB,DB,365
3,DB금융서비스,0.0,0.000000,0.000000,NaN,금융회사,DB,DB,365
4,DB금융서비스,1.0,1.000000,1.000000,DB엠앤에스,금융회사,DB,DB,365
5,DB금융서비스,99.0,0.000000,0.000000,NaN,금융회사,DB,DB,365
...,...,...,...,...,...,...,...,...,...
75,동부철구,1.0,0.497143,0.497143,DB하이텍,비금융회사,DB,DB,365
76,동부철구,99.0,0.502857,0.502857,NaN,비금융회사,DB,DB,365
77,코메,0.0,0.000000,0.000000,NaN,비금융회사,DB,DB,365
78,코메,1.0,1.000000,1.000000,DB메탈,비금융회사,DB,DB,365


# 코오롱

In [458]:
df = pd.read_excel('../data/ownership-status/raw/2021/코오롱.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [459]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [460]:
dfOwn

,firmname,type,own,own2,ownername
0,그린나래,1.0,1.000000,1.000000,코오롱글로텍
1,네이처브리지,1.0,1.000000,1.000000,코오롱글로벌
2,더블유파트너스,0.0,1.000000,1.000000,NaN
3,덕평랜드,1.0,0.510000,0.510000,코오롱글로벌
4,덕평랜드,99.0,0.490000,0.490000,NaN
...,...,...,...,...,...
60,테크비전,1.0,1.000000,1.000000,코오롱글로벌
61,퍼플아이오,1.0,0.649995,0.649995,코오롱인더스트리
62,퍼플아이오,99.0,0.350005,0.350005,NaN
63,한중문화타운,1.0,0.700000,0.700000,코오롱글로벌


In [461]:
grpname2 = '코오롱'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

kolon


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,그린나래,1.0,1.000000,1.000000,코오롱글로텍,비금융회사,코오롱,kolon,811
1,네이처브리지,1.0,1.000000,1.000000,코오롱글로벌,비금융회사,코오롱,kolon,811
2,더블유파트너스,0.0,1.000000,1.000000,NaN,비금융회사,코오롱,kolon,811
3,덕평랜드,1.0,0.510000,0.510000,코오롱글로벌,비금융회사,코오롱,kolon,811
4,덕평랜드,99.0,0.490000,0.490000,NaN,비금융회사,코오롱,kolon,811
...,...,...,...,...,...,...,...,...,...
60,테크비전,1.0,1.000000,1.000000,코오롱글로벌,비금융회사,코오롱,kolon,811
61,퍼플아이오,1.0,0.649995,0.649995,코오롱인더스트리,비금융회사,코오롱,kolon,811
62,퍼플아이오,99.0,0.350005,0.350005,NaN,비금융회사,코오롱,kolon,811
63,한중문화타운,1.0,0.700000,0.700000,코오롱글로벌,비금융회사,코오롱,kolon,811


# 한국타이어

In [462]:
df = pd.read_excel('../data/ownership-status/raw/2021/한국앤컴퍼니.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [463]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
농업회사법인장지       0.100000
신양월드레저         1.200000
한국타이어앤테크놀로지    1.056669
Name: own2, dtype: float64

In [464]:
df[df['소속회사명']=='농업회사법인장지']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
110,비금융회사,농업회사법인장지,동일인측,친족,배우자/혈족1촌,조현식,9000,45%,-,-,9000.0,45%,NaN,NaN
111,비금융회사,농업회사법인장지,동일인측,친족,배우자/혈족1촌,조현범,9000,45%,-,-,9000.0,45%,NaN,NaN
112,비금융회사,농업회사법인장지,동일인측,친족합계,친족합계,친족합계,18000,90%,-,-,18000.0,90%,NaN,NaN
113,비금융회사,농업회사법인장지,동일인측,동일인측합계,동일인측합계,동일인측합계,18000,90%,-,-,18000.0,90%,NaN,NaN
114,비금융회사,농업회사법인장지,기타주주,기타,기타,-,2000,10%,-,-,2000.0,10%,99.0,99.0
115,비금융회사,농업회사법인장지,총계,총계,총계,총계,20000,100%,-,-,20000.0,100%,NaN,NaN


In [465]:
df.loc[112, '동일인과의관계2'] = '친족'

In [466]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
신양월드레저         1.200000
한국타이어앤테크놀로지    1.056669
Name: own2, dtype: float64

In [467]:
df[df['소속회사명']=='신양월드레저']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
72,비금융회사,신양월드레저,동일인측,동일인,동일인,조양래,40000,20%,-,-,40000.0,20%,0.0,0.0
73,비금융회사,신양월드레저,동일인측,친족,배우자/혈족1촌,조현식,80000,40%,-,-,80000.0,40%,NaN,NaN
74,비금융회사,신양월드레저,동일인측,친족,배우자/혈족1촌,조현범,80000,40%,-,-,80000.0,40%,NaN,NaN
75,비금융회사,신양월드레저,동일인측,친족,친족합계,친족합계,200000,100%,-,-,200000.0,100%,0.0,0.0
76,비금융회사,신양월드레저,동일인측,동일인측합계,동일인측합계,동일인측합계,200000,100%,-,-,200000.0,100%,NaN,NaN
77,비금융회사,신양월드레저,총계,총계,총계,총계,200000,100%,-,-,200000.0,100%,NaN,NaN


In [468]:
df.loc[75, '합계주식수'] = 80000.0 * 2

In [469]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
한국타이어앤테크놀로지    1.056669
Name: own2, dtype: float64

In [470]:
df[df['소속회사명']=='한국타이어앤테크놀로지']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
17,비금융회사,한국타이어앤테크놀로지,동일인측,동일인,동일인,조양래,7019903,5.67%,-,-,7019903.0,5.67%,0.0,0.0
18,비금융회사,한국타이어앤테크놀로지,동일인측,친족,배우자/혈족1촌,홍문자,9119,0.01%,-,-,9119.0,0.01%,NaN,NaN
19,비금융회사,한국타이어앤테크놀로지,동일인측,친족,배우자/혈족1촌,조현범,2561241,2.07%,-,-,2561241.0,2.07%,NaN,NaN
20,비금융회사,한국타이어앤테크놀로지,동일인측,친족,배우자/혈족1촌,조현식,799244,0.65%,-,-,799244.0,0.65%,NaN,NaN
21,비금융회사,한국타이어앤테크놀로지,동일인측,친족,배우자/혈족1촌,조희원,884239,0.71%,-,-,884239.0,0.71%,NaN,NaN
22,비금융회사,한국타이어앤테크놀로지,동일인측,친족,배우자/혈족1촌,조희경,3366870,2.72%,-,-,3366870.0,2.72%,NaN,NaN
23,비금융회사,한국타이어앤테크놀로지,동일인측,친족,혈족2~4촌,조재형,15397,0.01%,-,-,15397.0,0.01%,NaN,NaN
24,비금융회사,한국타이어앤테크놀로지,동일인측,친족,혈족2~4촌,조유빈,15351,0.01%,-,-,15351.0,0.01%,NaN,NaN
25,비금융회사,한국타이어앤테크놀로지,동일인측,친족,혈족2~4촌,조재민,15302,0.01%,-,-,15302.0,0.01%,NaN,NaN
26,비금융회사,한국타이어앤테크놀로지,동일인측,친족,혈족2~4촌,조재완,15447,0.01%,-,-,15447.0,0.01%,NaN,NaN


In [471]:
df.loc[29, '합계주식수'] = 14754291.0 - 7019903.0

In [472]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [473]:
grpname2 = '한국타이어'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

hantire


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,농업회사법인장지,0.0,0.900000,0.900000,NaN,비금융회사,한국타이어,hantire,912
1,농업회사법인장지,99.0,0.100000,0.100000,NaN,비금융회사,한국타이어,hantire,912
2,리뉴에너지,99.0,1.000000,1.000000,NaN,비금융회사,한국타이어,hantire,912
3,모델솔루션,1.0,0.749999,0.749999,한국타이어앤테크놀로지,비금융회사,한국타이어,hantire,912
4,모델솔루션,99.0,0.250001,0.250001,NaN,비금융회사,한국타이어,hantire,912
5,세일환경,0.0,0.964385,0.964385,NaN,비금융회사,한국타이어,hantire,912
6,세일환경,99.0,0.035615,0.035615,NaN,비금융회사,한국타이어,hantire,912
7,신양관광개발,0.0,1.000000,1.000000,NaN,비금융회사,한국타이어,hantire,912
8,신양월드레저,0.0,1.000000,1.000000,NaN,비금융회사,한국타이어,hantire,912
9,아름덴티스트리,1.0,0.900000,0.900000,두원홀딩스,비금융회사,한국타이어,hantire,912


# 오씨아이

In [474]:
df = pd.read_excel('../data/ownership-status/raw/2021/OCI.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [475]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [476]:
dfOwn

,firmname,type,own,own2,ownername
0,디씨알이,0.0,0.000000,0.000000,NaN
1,디씨알이,1.0,1.000000,1.000000,오씨아이
2,디씨알이,99.0,0.000000,0.000000,NaN
3,비앤오바이오,0.0,0.000000,0.000000,NaN
4,비앤오바이오,1.0,0.500000,0.500000,OCI
5,비앤오바이오,99.0,0.500000,0.500000,NaN
6,에스지씨그린파워,0.0,0.000000,0.000000,NaN
7,에스지씨그린파워,1.0,0.950034,0.950034,에스지씨에너지
8,에스지씨그린파워,99.0,0.049966,0.049966,NaN
9,에스지씨디벨롭먼트,0.0,0.000000,0.000000,NaN


In [477]:
grpname2 = '오씨아이'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

dych


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,디씨알이,0.0,0.000000,0.000000,NaN,비 금 융 회 사,오씨아이,dych,378
1,디씨알이,1.0,1.000000,1.000000,오씨아이,비 금 융 회 사,오씨아이,dych,378
2,디씨알이,99.0,0.000000,0.000000,NaN,비 금 융 회 사,오씨아이,dych,378
3,비앤오바이오,0.0,0.000000,0.000000,NaN,비 금 융 회 사,오씨아이,dych,378
4,비앤오바이오,1.0,0.500000,0.500000,OCI,비 금 융 회 사,오씨아이,dych,378
5,비앤오바이오,99.0,0.500000,0.500000,NaN,비 금 융 회 사,오씨아이,dych,378
6,에스지씨그린파워,0.0,0.000000,0.000000,NaN,비 금 융 회 사,오씨아이,dych,378
7,에스지씨그린파워,1.0,0.950034,0.950034,에스지씨에너지,비 금 융 회 사,오씨아이,dych,378
8,에스지씨그린파워,99.0,0.049966,0.049966,NaN,비 금 융 회 사,오씨아이,dych,378
9,에스지씨디벨롭먼트,0.0,0.000000,0.000000,NaN,비 금 융 회 사,오씨아이,dych,378


# 이랜드

In [478]:
df = pd.read_excel('../data/ownership-status/raw/2021/이랜드월드.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [479]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [480]:
dfOwn

,firmname,type,own,own2,ownername
0,농업회사법인맛누리,0.0,0.0,0.1,NaN
1,농업회사법인맛누리,1.0,0.9,0.9,이랜드리테일
2,농업회사법인맛누리,99.0,0.0,0.0,NaN
3,농업회사법인켄싱턴팜,0.0,0.0,0.1,NaN
4,농업회사법인켄싱턴팜,1.0,0.9,0.9,이랜드파크
...,...,...,...,...,...
95,켄싱턴월드,1.0,1.0,1.0,이랜드파크
96,켄싱턴월드,99.0,0.0,0.0,NaN
97,켄싱턴평창,0.0,0.0,0.0,NaN
98,켄싱턴평창,1.0,1.0,1.0,"MicornesiaResort,Inc"


In [481]:
grpname2 = '이랜드'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

eland


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,농업회사법인맛누리,0.0,0.0,0.1,NaN,비금융회사,이랜드,eland,652
1,농업회사법인맛누리,1.0,0.9,0.9,이랜드리테일,비금융회사,이랜드,eland,652
2,농업회사법인맛누리,99.0,0.0,0.0,NaN,비금융회사,이랜드,eland,652
3,농업회사법인켄싱턴팜,0.0,0.0,0.1,NaN,비금융회사,이랜드,eland,652
4,농업회사법인켄싱턴팜,1.0,0.9,0.9,이랜드파크,비금융회사,이랜드,eland,652
...,...,...,...,...,...,...,...,...,...
95,켄싱턴월드,1.0,1.0,1.0,이랜드파크,비금융회사,이랜드,eland,652
96,켄싱턴월드,99.0,0.0,0.0,NaN,비금융회사,이랜드,eland,652
97,켄싱턴평창,0.0,0.0,0.0,NaN,비금융회사,이랜드,eland,652
98,켄싱턴평창,1.0,1.0,1.0,"MicornesiaResort,Inc",비금융회사,이랜드,eland,652


# 세아

In [482]:
df = pd.read_excel('../data/ownership-status/raw/2021/세아홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [483]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [484]:
dfOwn

,firmname,type,own,own2,ownername
0,동아스틸,0.0,0.000000,0.000000,NaN
1,동아스틸,1.0,0.976732,0.976732,에스에스아이케이
2,동아스틸,1.0,NaN,NaN,-
3,동아스틸,99.0,0.023268,0.023268,NaN
4,브이엔티지,0.0,0.000000,0.089684,NaN
...,...,...,...,...,...
107,이앤비원,99.0,0.000000,0.000000,NaN
108,"탈리다쿰구,이사야에이치피피",0.0,0.000000,0.000000,NaN
109,"탈리다쿰구,이사야에이치피피",1.0,0.841031,0.841031,에이치피피
110,"탈리다쿰구,이사야에이치피피",1.0,NaN,NaN,-


In [485]:
grpname2 = '세아'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

seah


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,동아스틸,0.0,0.000000,0.000000,NaN,비금융회사,세아,seah,474
1,동아스틸,1.0,0.976732,0.976732,에스에스아이케이,비금융회사,세아,seah,474
3,동아스틸,99.0,0.023268,0.023268,NaN,비금융회사,세아,seah,474
4,브이엔티지,0.0,0.000000,0.089684,NaN,비금융회사,세아,seah,474
5,브이엔티지,1.0,0.880349,0.880349,세아홀딩스,비금융회사,세아,seah,474
...,...,...,...,...,...,...,...,...,...
104,이앤비원,0.0,1.000000,1.000000,NaN,비금융회사,세아,seah,474
107,이앤비원,99.0,0.000000,0.000000,NaN,비금융회사,세아,seah,474
108,"탈리다쿰구,이사야에이치피피",0.0,0.000000,0.000000,NaN,비금융회사,세아,seah,474
109,"탈리다쿰구,이사야에이치피피",1.0,0.841031,0.841031,에이치피피,비금융회사,세아,seah,474


# 중흥건설

In [486]:
df = pd.read_excel('../data/ownership-status/raw/2021/중흥건설.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [487]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
세흥건설    0.861557
Name: own2, dtype: float64

In [488]:
grpname2 = '중흥건설'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

jungheung


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,나주관광개발,0.0,0.6216,0.6216,NaN,비금융회사,중흥건설,jungheung,743
1,나주관광개발,1.0,0.1784,0.1784,중흥건설,비금융회사,중흥건설,jungheung,743
2,나주관광개발,1.0,0.2000,0.2000,중흥토건,비금융회사,중흥건설,jungheung,743
3,나주관광개발,99.0,0.0000,0.0000,NaN,비금융회사,중흥건설,jungheung,743
4,남도일보,0.0,0.0000,0.0000,NaN,비금융회사,중흥건설,jungheung,743
...,...,...,...,...,...,...,...,...,...
119,헤럴드에듀,1.0,1.0000,1.0000,헤럴드,비금융회사,중흥건설,jungheung,743
120,헤럴드에듀,99.0,0.0000,0.0000,NaN,비금융회사,중흥건설,jungheung,743
121,헤럴드팝,0.0,0.0000,0.0000,NaN,비금융회사,중흥건설,jungheung,743
122,헤럴드팝,1.0,1.0000,1.0000,헤럴드,비금융회사,중흥건설,jungheung,743


# 태광

In [489]:
df = pd.read_excel('../data/ownership-status/raw/2021/태광산업.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [490]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
대신흥국제일호사모투자    0.000000
티시스            1.042299
Name: own2, dtype: float64

In [491]:
# 오기로 추정됨
df[df['소속회사명'] == '대신흥국제일호사모투자']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
246,금융회사,대신흥국제일호사모투자,동일인측,동일인,동일인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
247,금융회사,대신흥국제일호사모투자,동일인측,친족,배우자/혈족1촌,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248,금융회사,대신흥국제일호사모투자,동일인측,친족,혈족2~4촌,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249,금융회사,대신흥국제일호사모투자,동일인측,친족,혈족5~6촌,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,금융회사,대신흥국제일호사모투자,동일인측,친족,인척4촌이내,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251,금융회사,대신흥국제일호사모투자,동일인측,친족,친족합계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
252,금융회사,대신흥국제일호사모투자,동일인측,비영리법인,비영리법인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
253,금융회사,대신흥국제일호사모투자,동일인측,등기된임원,등기된임원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
254,금융회사,대신흥국제일호사모투자,동일인측,자기주식,자기주식,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
255,금융회사,대신흥국제일호사모투자,동일인측,계열회사,계열회사,흥국생명보험,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0


In [492]:
df[df['소속회사명'] == '티시스']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
117,비금융회사,티시스,동일인측,동일인,동일인,이호진,400000.0,4.23,-,-,400000.0,4.23,0.0,0.0
118,비금융회사,티시스,동일인측,친족,배우자/혈족1촌,이현준,1068586.0,11.30,-,-,1068586.0,11.30,NaN,NaN
119,비금융회사,티시스,동일인측,친족,배우자/혈족1촌,이현나,52263.0,0.55,-,-,52263.0,0.55,NaN,NaN
120,비금융회사,티시스,동일인측,친족,배우자/혈족1촌,신유나,52263.0,0.55,-,-,52263.0,0.55,NaN,NaN
121,비금융회사,티시스,동일인측,친족,혈족2~4촌,이원준,197000.0,2.08,-,-,197000.0,2.08,NaN,NaN
122,비금융회사,티시스,동일인측,친족,혈족5~6촌,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,비금융회사,티시스,동일인측,친족,인척4촌이내,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,비금융회사,티시스,동일인측,친족,친족합계,NaN,1770112.0,18.72,-,-,1770112.0,18.72,0.0,0.0
125,비금융회사,티시스,동일인측,비영리법인,비영리법인,학교법인일주세화학원,321567.0,3.40,-,-,321567.0,3.40,NaN,0.0
126,비금융회사,티시스,동일인측,등기된임원,등기된임원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [493]:
df.loc[124, '합계주식수'] = 1770112.0 - 400000.0

In [494]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
대신흥국제일호사모투자    0.0
Name: own2, dtype: float64

In [495]:
grpname2 = '태광'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

taekwang


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,고려저축은행,0.0,0.536515,0.536515,NaN,금융회사,태광,taekwang,841
1,고려저축은행,1.0,0.202415,0.202415,대한화섬,금융회사,태광,taekwang,841
2,고려저축은행,1.0,0.202415,0.202415,태광산업,금융회사,태광,taekwang,841
3,고려저축은행,1.0,0.058656,0.058656,흥국생명보험,금융회사,태광,taekwang,841
4,고려저축은행,99.0,0.000000,0.000000,NaN,금융회사,태광,taekwang,841
...,...,...,...,...,...,...,...,...,...
64,흥국증권,99.0,0.000000,0.000000,NaN,금융회사,태광,taekwang,841
65,흥국화재해상보험,0.0,0.000000,0.000000,NaN,금융회사,태광,taekwang,841
66,흥국화재해상보험,1.0,0.595555,0.595555,흥국생명보험,금융회사,태광,taekwang,841
67,흥국화재해상보험,1.0,0.196311,0.196311,태광산업,금융회사,태광,taekwang,841


# 동원

In [496]:
df = pd.read_excel('../data/ownership-status/raw/2021/동원엔터프라이즈.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [497]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
동원산업    0.994912
Name: own2, dtype: float64

In [498]:
grpname2 = '동원'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

dongwon


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,남춘천산업단지개발,1.0,4.565926e-01,0.456593,동원건설산업,비금융회사,동원,dongwon,381
1,남춘천산업단지개발,99.0,5.434074e-01,0.543407,NaN,비금융회사,동원,dongwon,381
2,농업회사법인어석,0.0,9.305867e-02,0.101151,NaN,비금융회사,동원,dongwon,381
3,농업회사법인어석,1.0,8.988493e-01,0.898849,동원산업,비금융회사,동원,dongwon,381
4,동원건설산업,1.0,1.000000e+00,1.000000,동원엔터프라이즈,비금융회사,동원,dongwon,381
5,동원건설산업,99.0,0.000000e+00,0.000000,NaN,비금융회사,동원,dongwon,381
6,동원디어푸드,1.0,1.000000e+00,1.000000,동원에프앤비,비금융회사,동원,dongwon,381
7,동원디어푸드,99.0,0.000000e+00,0.000000,NaN,비금융회사,동원,dongwon,381
8,동원로엑스,1.0,1.000000e+00,1.000000,동원산업,비금융회사,동원,dongwon,381
9,동원로엑스,99.0,0.000000e+00,0.000000,NaN,비금융회사,동원,dongwon,381


# 한라

In [499]:
df = pd.read_excel('../data/ownership-status/raw/2021/한라홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [500]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [501]:
dfOwn

,firmname,type,own,own2,ownername
0,로터스프라이빗에쿼티,0.0,1.000000,1.000000,NaN
1,만도,0.0,0.000070,0.002943,NaN
2,만도,1.0,0.302473,0.302473,한라홀딩스
3,만도,99.0,0.694585,0.694585,NaN
4,만도브로제,1.0,0.500000,0.500000,만도
5,만도브로제,99.0,0.500000,0.500000,NaN
6,만도차이나홀딩스,1.0,1.000000,1.000000,만도
7,만도헬라일렉트로닉스,1.0,1.000000,1.000000,만도
8,목포신항만운영,1.0,1.000000,1.000000,한라
9,배곧신도시지역특성화타운,1.0,0.950000,0.950000,한라


In [502]:
grpname2 = '한라'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

halla


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,로터스프라이빗에쿼티,0.0,1.000000,1.000000,NaN,금융회사,한라,halla,920
1,만도,0.0,0.000070,0.002943,NaN,비금융회사,한라,halla,920
2,만도,1.0,0.302473,0.302473,한라홀딩스,비금융회사,한라,halla,920
3,만도,99.0,0.694585,0.694585,NaN,비금융회사,한라,halla,920
4,만도브로제,1.0,0.500000,0.500000,만도,비금융회사,한라,halla,920
5,만도브로제,99.0,0.500000,0.500000,NaN,비금융회사,한라,halla,920
6,만도차이나홀딩스,1.0,1.000000,1.000000,만도,비금융회사,한라,halla,920
7,만도헬라일렉트로닉스,1.0,1.000000,1.000000,만도,비금융회사,한라,halla,920
8,목포신항만운영,1.0,1.000000,1.000000,한라,비금융회사,한라,halla,920
9,배곧신도시지역특성화타운,1.0,0.950000,0.950000,한라,비금융회사,한라,halla,920


# 아모레퍼시픽

In [503]:
df = pd.read_excel('../data/ownership-status/raw/2021/아모레퍼시픽그룹.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [504]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
이니스프리        1.0
코어테크놀로지    800.2
Name: own2, dtype: float64

In [505]:
df[df['소속회사명'] == '코어테크놀로지']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
186,비금융회사,코어테크놀로지,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
187,비금융회사,코어테크놀로지,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
188,비금융회사,코어테크놀로지,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
189,비금융회사,코어테크놀로지,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
190,비금융회사,코어테크놀로지,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
191,비금융회사,코어테크놀로지,동일인측,친족,친족합계,-,-,-,-,-,NaN,-,0.0,0.0
192,비금융회사,코어테크놀로지,동일인측,비영리법인,비영리법인,-,-,-,-,-,NaN,-,NaN,0.0
193,비금융회사,코어테크놀로지,동일인측,등기된임원,등기된임원,-,-,-,-,-,NaN,-,NaN,0.0
194,비금융회사,코어테크놀로지,동일인측,자기주식,자기주식,-,-,-,-,-,NaN,-,NaN,0.0
195,비금융회사,코어테크놀로지,동일인측,계열회사(국내+해외),계열회사(국내+해외),아모레퍼시픽,200000,80.00,-,-,200000.0,8000,1.0,1.0


In [506]:
df.loc[199, '합계주식수'] = 50000
df.loc[200, '합계주식수'] = 250000

In [507]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
이니스프리    1.0
Name: own2, dtype: float64

In [508]:
grpname2 = '아모레퍼시픽'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

amore


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,그린파트너즈,0.0,0.000000,0.000000,NaN,비금융회사,아모레퍼시픽,amore,842
1,그린파트너즈,1.0,1.000000,1.000000,오설록,비금융회사,아모레퍼시픽,amore,842
2,그린파트너즈,99.0,0.000000,0.000000,NaN,비금융회사,아모레퍼시픽,amore,842
3,농업회사법인오설록농장,0.0,0.000000,0.015740,NaN,비금융회사,아모레퍼시픽,amore,842
4,농업회사법인오설록농장,1.0,0.983768,0.983768,아모레퍼시픽그룹,비금융회사,아모레퍼시픽,amore,842
5,농업회사법인오설록농장,99.0,0.000492,0.000492,NaN,비금융회사,아모레퍼시픽,amore,842
6,아모레퍼시픽,0.0,0.092855,0.111441,NaN,비금융회사,아모레퍼시픽,amore,842
7,아모레퍼시픽,1.0,0.336421,0.336421,아모레퍼시픽그룹,비금융회사,아모레퍼시픽,amore,842
8,아모레퍼시픽,99.0,0.552138,0.552138,NaN,비금융회사,아모레퍼시픽,amore,842
9,아모레퍼시픽그룹,0.0,0.526719,0.639817,NaN,비금융회사,아모레퍼시픽,amore,842


# 아이엠엠인베스트먼트

In [509]:
df = pd.read_excel('../data/ownership-status/raw/2021/아이엠엠인베스트먼트.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [510]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
농업회사법인팜에이트          0.521200
아이엠엠인베스트먼트          1.000000
아이엠엠인프라제4호사모투자      1.000000
아이엠엠인프라제5의1호사모투자    1.000000
오하임아이엔티             0.583097
이미인                 0.761971
Name: own2, dtype: float64

In [511]:
df[df['소속회사명'] == '농업회사법인팜에이트']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
1954,비금융회사,농업회사법인팜에이트,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
1955,비금융회사,농업회사법인팜에이트,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
1956,비금융회사,농업회사법인팜에이트,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
1957,비금융회사,농업회사법인팜에이트,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
1958,비금융회사,농업회사법인팜에이트,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
1959,비금융회사,농업회사법인팜에이트,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
1960,비금융회사,농업회사법인팜에이트,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
1961,비금융회사,농업회사법인팜에이트,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
1962,비금융회사,농업회사법인팜에이트,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
1963,비금융회사,농업회사법인팜에이트,동일인측,친족,친족합계,/,-,-,-,-,NaN,-,0.0,0.0


In [512]:
df[df['소속회사명'] == '오하임아이엔티']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
1865,비금융회사,오하임아이엔티,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
1866,비금융회사,오하임아이엔티,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
1867,비금융회사,오하임아이엔티,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
1868,비금융회사,오하임아이엔티,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
1869,비금융회사,오하임아이엔티,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
1870,비금융회사,오하임아이엔티,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
1871,비금융회사,오하임아이엔티,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
1872,비금융회사,오하임아이엔티,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
1873,비금융회사,오하임아이엔티,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
1874,비금융회사,오하임아이엔티,동일인측,친족,친족합계,/,-,-,-,-,NaN,-,0.0,0.0


In [513]:
df[df['소속회사명'] == '이미인']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
1888,비금융회사,이미인,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
1889,비금융회사,이미인,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
1890,비금융회사,이미인,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
1891,비금융회사,이미인,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
1892,비금융회사,이미인,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
1893,비금융회사,이미인,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
1894,비금융회사,이미인,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
1895,비금융회사,이미인,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
1896,비금융회사,이미인,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
1897,비금융회사,이미인,동일인측,친족,친족합계,/,-,-,-,-,NaN,-,0.0,0.0


In [514]:
df.loc[1900, '합계주식수'] = 4757

In [515]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
농업회사법인팜에이트          0.521200
아이엠엠인베스트먼트          1.000000
아이엠엠인프라제4호사모투자      1.000000
아이엠엠인프라제5의1호사모투자    1.000000
오하임아이엔티             0.583097
이미인                 1.000000
Name: own2, dtype: float64

In [516]:
grpname2 = 'IMM인베스트먼트'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

IMM


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,강동냉장,0.0,0.000000,0.000000,NaN,비금융회사,IMM인베스트먼트,IMM,1210330
1,강동냉장,1.0,1.000000,1.000000,브루넬,비금융회사,IMM인베스트먼트,IMM,1210330
3,강동냉장,99.0,0.000000,0.000000,NaN,비금융회사,IMM인베스트먼트,IMM,1210330
4,강화클린,0.0,0.000000,0.000000,NaN,비금융회사,IMM인베스트먼트,IMM,1210330
5,강화클린,1.0,1.000000,1.000000,그린강화,비금융회사,IMM인베스트먼트,IMM,1210330
...,...,...,...,...,...,...,...,...,...
325,하논사모투자,1.0,0.090476,0.090476,아이엠엠인베스트먼트,금융회사,IMM인베스트먼트,IMM,1210330
326,하논사모투자,99.0,0.909524,0.909524,NaN,금융회사,IMM인베스트먼트,IMM,1210330
327,한국환경개발,0.0,0.000000,0.000000,NaN,비금융회사,IMM인베스트먼트,IMM,1210330
328,한국환경개발,1.0,1.000000,1.000000,에코매니지먼트코리아홀딩스,비금융회사,IMM인베스트먼트,IMM,1210330


# 삼천리

In [517]:
df = pd.read_excel('../data/ownership-status/raw/2021/삼천리.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [518]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
삼천리    0.961165
Name: own2, dtype: float64

In [519]:
df[df['소속회사명'] == '삼천리']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
0,비금융회사,삼천리,동일인측,동일인,동일인,이만득,338390,8.34,-,-,338390.0,8.34,0.0,0.0
1,비금융회사,삼천리,동일인측,친족,배우자/혈족1촌,이은희,27045,0.67,-,-,27045.0,0.67,NaN,NaN
2,비금융회사,삼천리,동일인측,친족,배우자/혈족1촌,이은남,27045,0.67,-,-,27045.0,0.67,NaN,NaN
3,비금융회사,삼천리,동일인측,친족,배우자/혈족1촌,이은선YiEunSun,27045,0.67,NaN,-,27045.0,0.67,NaN,NaN
4,비금융회사,삼천리,동일인측,친족,혈족2~4촌,이은백,372070,9.18,-,-,372070.0,9.18,NaN,NaN
5,비금융회사,삼천리,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
6,비금융회사,삼천리,동일인측,친족,인척4촌이내,-,-,NaN,-,-,NaN,NaN,NaN,NaN
7,비금융회사,삼천리,동일인측,친족,친족합계,/,453205,11.18,-,-,453205.0,11.18,0.0,0.0
8,비금융회사,삼천리,동일인측,비영리법인,비영리법인,NaN,NaN,NaN,-,-,NaN,NaN,NaN,0.0
9,비금융회사,삼천리,동일인측,등기된임원,등기된임원,길형도,250,0.01,-,-,250.0,0.01,NaN,0.0


In [520]:
grpname2 = '삼천리'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

samchully


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,가시태양광발전,0.0,0.00,0.00,NaN,비금융회사,삼천리,samchully,521
1,가시태양광발전,1.0,1.00,1.00,아폴로,비금융회사,삼천리,samchully,521
2,가시태양광발전,99.0,0.00,0.00,NaN,비금융회사,삼천리,samchully,521
3,동강풍력발전,0.0,0.00,0.00,NaN,비금융회사,삼천리,samchully,521
4,동강풍력발전,1.0,1.00,1.00,유니슨,비금융회사,삼천리,samchully,521
...,...,...,...,...,...,...,...,...,...
118,해런솔라,1.0,1.00,1.00,아폴로,비금융회사,삼천리,samchully,521
119,해런솔라,99.0,0.00,0.00,NaN,비금융회사,삼천리,samchully,521
120,휴세스,0.0,0.00,0.00,NaN,비금융회사,삼천리,samchully,521
121,휴세스,1.0,0.51,0.51,삼천리,비금융회사,삼천리,samchully,521


# 금호석유화학

In [521]:
df = pd.read_excel('../data/ownership-status/raw/2021/금호석유화학.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [522]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [523]:
dfOwn

,firmname,type,own,own2,ownername
0,강원학교태양광,1.0,1.000000,1.000000,코리아에너지발전소
1,금호개발상사,1.0,1.000000,1.000000,금호피앤비화학
2,금호리조트,1.0,0.667221,0.667221,금호석유화학
3,금호리조트,1.0,0.332779,0.332779,금호피앤비화학
4,금호미쓰이화학,1.0,0.500000,0.500000,금호석유화학
5,금호미쓰이화학,99.0,0.500000,0.500000,NaN
6,금호석유화학,0.0,0.226999,0.394760,NaN
7,금호석유화학,99.0,0.605240,0.605240,NaN
8,금호티앤엘,1.0,1.000000,1.000000,금호석유화학
9,금호폴리켐,1.0,0.500000,0.500000,금호석유화학


In [524]:
grpname2 = '금호석유화학'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

kpc


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,강원학교태양광,1.0,1.000000,1.000000,코리아에너지발전소,비금융회사,금호석유화학,kpc,11140
1,금호개발상사,1.0,1.000000,1.000000,금호피앤비화학,비금융회사,금호석유화학,kpc,11140
2,금호리조트,1.0,0.667221,0.667221,금호석유화학,비금융회사,금호석유화학,kpc,11140
3,금호리조트,1.0,0.332779,0.332779,금호피앤비화학,비금융회사,금호석유화학,kpc,11140
4,금호미쓰이화학,1.0,0.500000,0.500000,금호석유화학,비금융회사,금호석유화학,kpc,11140
5,금호미쓰이화학,99.0,0.500000,0.500000,NaN,비금융회사,금호석유화학,kpc,11140
6,금호석유화학,0.0,0.226999,0.394760,NaN,비금융회사,금호석유화학,kpc,11140
7,금호석유화학,99.0,0.605240,0.605240,NaN,비금융회사,금호석유화학,kpc,11140
8,금호티앤엘,1.0,1.000000,1.000000,금호석유화학,비금융회사,금호석유화학,kpc,11140
9,금호폴리켐,1.0,0.500000,0.500000,금호석유화학,비금융회사,금호석유화학,kpc,11140


# 다우키움

In [525]:
df = pd.read_excel('../data/ownership-status/raw/2021/키움증권.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [526]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
포스코우리이아이지글로벌사모투자    1.0
한국전력우리스프랏글로벌사모투자    1.0
Name: own2, dtype: float64

In [527]:
grpname2 = '다우키움'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

daou


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,게티이미지코리아,0.0,0.000000,0.000000,NaN,비금융회사,다우키움,daou,291
1,게티이미지코리아,1.0,0.500000,0.500000,이매진스,비금융회사,다우키움,daou,291
3,게티이미지코리아,99.0,0.500000,0.500000,NaN,비금융회사,다우키움,daou,291
4,글로벌강소기업키움엠앤에이전략창업벤처전문사모투자,0.0,0.000000,0.000000,NaN,금융회사,다우키움,daou,291
5,글로벌강소기업키움엠앤에이전략창업벤처전문사모투자,1.0,0.200000,0.200000,키움증권,금융회사,다우키움,daou,291
...,...,...,...,...,...,...,...,...,...
181,한국전력우리스프랏글로벌해외자원개발제3차,1.0,1.000000,1.000000,한국전력우리스프랏글로벌해외자원개발사모투자,금융회사,다우키움,daou,291
183,한국전력우리스프랏글로벌해외자원개발제3차,99.0,0.000000,0.000000,NaN,금융회사,다우키움,daou,291
184,한국정보인증,0.0,0.000000,0.000000,NaN,비금융회사,다우키움,daou,291
185,한국정보인증,1.0,0.374749,0.374749,다우기술,비금융회사,다우키움,daou,291


# 장금상선

In [528]:
df = pd.read_excel('../data/ownership-status/raw/2021/장금상선.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [529]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
장금상선    0.179348
Name: own2, dtype: float64

In [530]:
df[df['소속회사명'] == '장금상선']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
0,비금융회사,장금상선,동일인측,동일인,동일인,정태순,62000,16.85%,-,-,62000.0,16.85%,0.0,0.0
1,비금융회사,장금상선,동일인측,친족,배우자/,-,-,-,-,-,NaN,-,NaN,NaN
2,비금융회사,장금상선,동일인측,친족,혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
3,비금융회사,장금상선,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
4,비금융회사,장금상선,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
5,비금융회사,장금상선,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
6,비금융회사,장금상선,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
7,비금융회사,장금상선,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
8,비금융회사,장금상선,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
9,비금융회사,장금상선,동일인측,친족,친족합계,/,-,-,-,-,NaN,-,0.0,0.0


In [531]:
grpname2 = '장금상선'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

sinokor


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,경평물류,0.0,0.000000,0.000000,NaN,비금융회사,장금상선,sinokor,2102509
1,경평물류,1.0,0.650000,0.650000,장금상선,비금융회사,장금상선,sinokor,2102509
2,경평물류,1.0,0.050000,0.050000,국양로지텍,비금융회사,장금상선,sinokor,2102509
3,경평물류,99.0,0.300000,0.300000,NaN,비금융회사,장금상선,sinokor,2102509
4,광양항서부컨테이너터미널,0.0,0.000000,0.000000,NaN,비금융회사,장금상선,sinokor,2102509
5,광양항서부컨테이너터미널,1.0,0.700000,0.700000,장금상선,비금융회사,장금상선,sinokor,2102509
7,광양항서부컨테이너터미널,99.0,0.300000,0.300000,NaN,비금융회사,장금상선,sinokor,2102509
8,국양로지텍,0.0,0.120303,0.120303,NaN,비금융회사,장금상선,sinokor,2102509
9,국양로지텍,1.0,0.879697,0.879697,장금상선,비금융회사,장금상선,sinokor,2102509
11,국양로지텍,99.0,0.000000,0.000000,NaN,비금융회사,장금상선,sinokor,2102509


# 동국제강

In [532]:
df = pd.read_excel('../data/ownership-status/raw/2021/동국제강.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [533]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
인터지스웅동센터    0.1009
Name: own2, dtype: float64

In [534]:
df[df['소속회사명'] == '인터지스웅동센터']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
59,비금융회사,인터지스웅동센터,동일인측,계열회사(국내+해외),계열회사(국내+해외),인터지스,720000,90.00,-,-,720.0,90.00,1.0,1.0
60,비금융회사,인터지스웅동센터,동일인측,동일인측합계,동일인측합계,/,720000,90.00,-,-,720.0,90.00,NaN,NaN
61,비금융회사,인터지스웅동센터,기타주주,기타,기타,/,80000,10.00,-,-,80000.0,10.00,99.0,99.0
62,비금융회사,인터지스웅동센터,총계,총계,총계,/,800000,100,-,-,800000.0,100,NaN,NaN


In [535]:
df.loc[59, '합계주식수'] = 720000

In [536]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [537]:
grpname2 = '동국제강'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

dongkuk


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,가온에스티,1.0,0.642478,0.642478,동국제강,비금융회사,동국제강,dongkuk,364
1,가온에스티,99.0,0.357522,0.357522,NaN,비금융회사,동국제강,dongkuk,364
2,당진고대부두운영,1.0,0.212195,0.212195,동국제강,비금융회사,동국제강,dongkuk,364
3,당진고대부두운영,1.0,0.487805,0.487805,인터지스,비금융회사,동국제강,dongkuk,364
4,당진고대부두운영,99.0,0.300000,0.300000,NaN,비금융회사,동국제강,dongkuk,364
5,동국시스템즈,1.0,1.000000,1.000000,동국제강,비금융회사,동국제강,dongkuk,364
6,동국제강,0.0,0.255332,0.281995,NaN,비금융회사,동국제강,dongkuk,364
7,동국제강,99.0,0.718005,0.718005,NaN,비금융회사,동국제강,dongkuk,364
8,아이앤케이신항만,1.0,0.500000,0.500000,인터지스,비금융회사,동국제강,dongkuk,364
9,아이앤케이신항만,99.0,0.500000,0.500000,NaN,비금융회사,동국제강,dongkuk,364


# 애경

In [538]:
df = pd.read_excel('../data/ownership-status/raw/2021/AK홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [539]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
서림          2.000000
에이케이에스앤디    0.999739
Name: own2, dtype: float64

In [540]:
df[df['소속회사명'] == '서림']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
685,비금융회사,서림,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
686,비금융회사,서림,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
687,비금융회사,서림,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
688,비금융회사,서림,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
689,비금융회사,서림,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
690,비금융회사,서림,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
691,비금융회사,서림,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
692,비금융회사,서림,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
693,비금융회사,서림,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
694,비금융회사,서림,동일인측,친족,친족합계,/,-,-,-,-,NaN,-,0.0,0.0


In [541]:
df.loc[705, '동일인과의관계'] = '총계'

In [542]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
에이케이에스앤디    0.999739
Name: own2, dtype: float64

In [543]:
grpname2 = '애경'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

AK


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,마포애경타운,0.0,0.000000,0.000000,NaN,비금융회사,애경,AK,615
1,마포애경타운,1.0,0.914400,0.914400,에이케이에스앤디,비금융회사,애경,AK,615
2,마포애경타운,99.0,0.085600,0.085600,NaN,비금융회사,애경,AK,615
3,모두락,0.0,0.000000,0.000000,NaN,비금융회사,애경,AK,615
4,모두락,1.0,1.000000,1.000000,제주항공,비금융회사,애경,AK,615
...,...,...,...,...,...,...,...,...,...
139,코스파,1.0,0.100000,0.100000,에이케이아이에스,비금융회사,애경,AK,615
140,코스파,99.0,0.500000,0.500000,NaN,비금융회사,애경,AK,615
141,퍼시픽제3호전문사모부동산투자,0.0,0.000000,0.000000,NaN,비금융회사,애경,AK,615
142,퍼시픽제3호전문사모부동산투자,1.0,0.998586,0.998586,제주항공,비금융회사,애경,AK,615


# 반도홀딩스

In [544]:
df = pd.read_excel('../data/ownership-status/raw/2021/반도홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [545]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
반도레저    0.1
Name: own2, dtype: float64

In [546]:
df[df['소속회사명'] == '반도레저']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
185,비금융회사,반도레저,동일인측,동일인,동일인,권홍사,20000,10.00%,-,-,20000.0,10.00%,0.0,0.0
186,비금융회사,반도레저,동일인측,친족,배우자/혈족1촌,권재현,159800,79.90%,-,-,159800.0,79.90%,NaN,NaN
187,비금융회사,반도레저,동일인측,친족,배우자/혈족1촌,유성애,20000,10.00%,-,-,20000.0,10.00%,NaN,NaN
188,비금융회사,반도레저,동일인측,친족,배우자/혈족1촌,신동철,200,0.10%,-,-,200.0,0.10%,NaN,NaN
189,비금융회사,반도레저,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
190,비금융회사,반도레저,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
191,비금융회사,반도레저,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
192,비금융회사,반도레저,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
193,비금융회사,반도레저,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
194,비금융회사,반도레저,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN


In [547]:
df.loc[195, '합계주식수'] = 180000.0

In [548]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [549]:
grpname2 = '반도홀딩스'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

bando


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,대영개발,0.0,0.0,0.0,NaN,비금융회사,반도홀딩스,bando,22061
1,대영개발,1.0,1.0,1.0,반도종합건설,비금융회사,반도홀딩스,bando,22061
3,대영개발,99.0,0.0,0.0,NaN,비금융회사,반도홀딩스,bando,22061
4,대창개발,0.0,0.0,0.0,NaN,비금융회사,반도홀딩스,bando,22061
5,대창개발,1.0,1.0,1.0,반도종합건설,비금융회사,반도홀딩스,bando,22061
...,...,...,...,...,...,...,...,...,...
106,호법포레,1.0,0.3,0.3,반도개발,비금융회사,반도홀딩스,bando,22061
107,호법포레,99.0,0.0,0.0,NaN,비금융회사,반도홀딩스,bando,22061
108,화인개발,0.0,0.0,0.0,NaN,비금융회사,반도홀딩스,bando,22061
109,화인개발,1.0,1.0,1.0,반도건설,비금융회사,반도홀딩스,bando,22061


# 유진

In [550]:
df = pd.read_excel('../data/ownership-status/raw/2021/유진기업.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [551]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
동양      0.941753
천안기업    1.108728
클린메디    2.000000
Name: own2, dtype: float64

In [552]:
df[df['소속회사명']=='동양']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
43,비금융회사,동양,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
44,비금융회사,동양,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
45,비금융회사,동양,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
46,비금융회사,동양,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
47,비금융회사,동양,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
48,비금융회사,동양,동일인측,친족,친족합계,/,-,-,-,-,NaN,-,0.0,0.0
49,비금융회사,동양,동일인측,비영리법인,비영리법인,-,-,-,-,-,NaN,-,NaN,0.0
50,비금융회사,동양,동일인측,등기된임원,등기된임원,정진학,50000,0.02,-,-,50000.0,0.02,NaN,0.0
51,비금융회사,동양,동일인측,자기주식,자기주식,동양,40512000,16.97,171981,16.76,40683981.0,16.97,NaN,0.0
52,비금융회사,동양,동일인측,계열회사(국내+해외),계열회사(국내+해외),유진기업,56768592,23.78,-,-,56768592.0,23.78,1.0,1.0


In [553]:
df[df['소속회사명']=='클린메디']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
791,금융회사,클린메디,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
792,금융회사,클린메디,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
793,금융회사,클린메디,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
794,금융회사,클린메디,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
795,금융회사,클린메디,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
796,금융회사,클린메디,동일인측,친족,친족합계,/,-,-,-,-,NaN,-,0.0,0.0
797,금융회사,클린메디,동일인측,비영리법인,비영리법인,-,-,-,-,-,NaN,-,NaN,0.0
798,금융회사,클린메디,동일인측,등기된임원,등기된임원,-,-,-,-,-,NaN,-,NaN,0.0
799,금융회사,클린메디,동일인측,자기주식,자기주식,-,-,-,-,-,NaN,-,NaN,0.0
800,금융회사,클린메디,동일인측,계열회사(국내+해외),계열회사(국내+해외),인프라이니셔티브1호사모투자,23550000,100.00,-,-,23550000.0,100.00,1.0,1.0


In [554]:
df.loc[803, '합계주식수'] = 0

In [555]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
동양      0.941753
천안기업    1.108728
Name: own2, dtype: float64

In [556]:
df[df['소속회사명']=='천안기업']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
521,비금융회사,천안기업,동일인측,동일인,동일인,유경선,189422,11.56,-,-,189422.0,11.56,0.0,0.0
522,비금융회사,천안기업,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
523,비금융회사,천안기업,동일인측,친족,혈족2~4촌,유창수,123957,7.56,-,-,123957.0,7.56,NaN,NaN
524,비금융회사,천안기업,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
525,비금융회사,천안기업,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
526,비금융회사,천안기업,동일인측,친족,친족합계,/,302157,19.12,-,-,302157.0,19.12,0.0,0.0
527,비금융회사,천안기업,동일인측,비영리법인,비영리법인,-,-,-,-,-,NaN,-,NaN,0.0
528,비금융회사,천안기업,동일인측,등기된임원,등기된임원,-,-,-,-,-,NaN,-,NaN,0.0
529,비금융회사,천안기업,동일인측,자기주식,자기주식,-,-,-,-,-,NaN,-,NaN,0.0
530,비금융회사,천안기업,동일인측,계열회사(국내+해외),계열회사(국내+해외),유진기업,1325569,80.88,-,-,1325569.0,80.88,1.0,1.0


In [557]:
df.loc[526, '합계주식수'] = 123957.0

In [558]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
동양    0.941753
Name: own2, dtype: float64

In [559]:
grpname2 = '유진'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

eugenes


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,굿앤파트너스,0.0,0.00,0.00,NaN,비금융회사,유진,eugenes,653
1,굿앤파트너스,1.0,1.00,1.00,유진로지스틱스,비금융회사,유진,eugenes,653
2,굿앤파트너스,99.0,0.00,0.00,NaN,비금융회사,유진,eugenes,653
3,나눔로또,0.0,0.00,0.00,NaN,비금융회사,유진,eugenes,653
4,나눔로또,1.0,1.00,1.00,유진기업,비금융회사,유진,eugenes,653
...,...,...,...,...,...,...,...,...,...
164,현대개발,0.0,0.00,0.00,NaN,비금융회사,유진,eugenes,653
165,현대개발,1.0,1.00,1.00,유진기업,비금융회사,유진,eugenes,653
166,현대개발,99.0,0.00,0.00,NaN,비금융회사,유진,eugenes,653
167,호남아스콘,0.0,0.45,0.45,NaN,비금융회사,유진,eugenes,653


# 하이트진로

In [560]:
df = pd.read_excel('../data/ownership-status/raw/2021/하이트진로홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [561]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [562]:
dfOwn

,firmname,type,own,own2,ownername
0,강원물류,0.0,0.000000,0.000000,NaN
1,강원물류,1.0,1.000000,1.000000,하이트진로
2,강원물류,99.0,0.000000,0.000000,NaN
3,농업회사법인팜컬쳐,0.0,0.000000,0.100000,NaN
4,농업회사법인팜컬쳐,1.0,0.900000,0.900000,하이트진로
5,농업회사법인팜컬쳐,99.0,0.000000,0.000000,NaN
6,대우컴바인,0.0,1.000000,1.000000,NaN
7,대우컴바인,1.0,NaN,NaN,-
8,대우컴바인,99.0,0.000000,0.000000,NaN
9,대우패키지,0.0,1.000000,1.000000,NaN


In [563]:
grpname2 = '하이트진로'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

hite


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,강원물류,0.0,0.000000,0.000000,NaN,비금융회사,하이트진로,hite,732
1,강원물류,1.0,1.000000,1.000000,하이트진로,비금융회사,하이트진로,hite,732
2,강원물류,99.0,0.000000,0.000000,NaN,비금융회사,하이트진로,hite,732
3,농업회사법인팜컬쳐,0.0,0.000000,0.100000,NaN,비금융회사,하이트진로,hite,732
4,농업회사법인팜컬쳐,1.0,0.900000,0.900000,하이트진로,비금융회사,하이트진로,hite,732
5,농업회사법인팜컬쳐,99.0,0.000000,0.000000,NaN,비금융회사,하이트진로,hite,732
6,대우컴바인,0.0,1.000000,1.000000,NaN,비금융회사,하이트진로,hite,732
8,대우컴바인,99.0,0.000000,0.000000,NaN,비금융회사,하이트진로,hite,732
9,대우패키지,0.0,1.000000,1.000000,NaN,비금융회사,하이트진로,hite,732
11,대우패키지,99.0,0.000000,0.000000,NaN,비금융회사,하이트진로,hite,732


# 삼양

In [564]:
df = pd.read_excel('../data/ownership-status/raw/2021/삼양홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [565]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
삼양에프앤비    2.0
삼양패키징     4.0
Name: own2, dtype: float64

In [566]:
df[df['소속회사명']=='삼양에프앤비']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
158,비금융회사,삼양에프앤비,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
159,비금융회사,삼양에프앤비,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
160,비금융회사,삼양에프앤비,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
161,비금융회사,삼양에프앤비,동일인측,친족,친족합계,-,-,-,-,-,NaN,-,0.0,0.0
162,비금융회사,삼양에프앤비,동일인측,비영리법인,비영리법인,-,-,-,-,-,NaN,-,NaN,0.0
163,비금융회사,삼양에프앤비,동일인측,자기주식,자기주식,-,-,-,-,-,NaN,-,NaN,0.0
164,비금융회사,삼양에프앤비,동일인측,계열회사(국내+해외),계열회사(국내+해외),삼양홀딩스,1876836,100.00,-,-,1876836.0,100.00,1.0,1.0
165,비금융회사,삼양에프앤비,동일인측,기타동일인관련자,기타동일인관련자,-,-,-,-,-,NaN,-,NaN,NaN
166,비금융회사,삼양에프앤비,동일인측,동일인측합계,동일인측합계,/,1876836,100.00,-,-,1876836.0,100.00,NaN,NaN
167,비금융회사,삼양에프앤비,기타주주,동일인측이아닌최다주주,동일인측이아닌최다주주,-,-,-,-,-,NaN,-,99.0,99.0


In [567]:
df.loc[168, '합계주식수'] = 0

In [568]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
삼양패키징    4.0
Name: own2, dtype: float64

In [569]:
df[df['소속회사명'] == '삼양패키징']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
55,비금융회사,삼양패키징,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
56,비금융회사,삼양패키징,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
57,비금융회사,삼양패키징,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
58,비금융회사,삼양패키징,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
59,비금융회사,삼양패키징,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
60,비금융회사,삼양패키징,동일인측,친족,친족합계,/,-,-,-,-,NaN,-,0.0,0.0
61,비금융회사,삼양패키징,동일인측,비영리법인,비영리법인,-,-,-,-,-,NaN,-,NaN,0.0
62,비금융회사,삼양패키징,동일인측,등기된임원,등기된임원,-,-,-,-,-,NaN,-,NaN,0.0
63,비금융회사,삼양패키징,동일인측,자기주식,자기주식,-,-,-,-,-,NaN,-,NaN,0.0
64,비금융회사,삼양패키징,동일인측,계열회사(국내+해외),계열회사(국내+해외),삼양사,9378467,66.00,-,-,9378467.0,66.00,1.0,1.0


In [570]:
df.loc[69, '합계주식수'] = 0
df.loc[70, '합계주식수'] = 14209804.0

In [571]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
삼양패키징   NaN
Name: own2, dtype: float64

In [572]:
dfOwn[dfOwn['firmname'] == '삼양패키징']

,firmname,type,own,own2,ownername
15,삼양패키징,0.0,0.00,0.00,NaN
16,삼양패키징,1.0,inf,inf,삼양사
17,삼양패키징,1.0,0.66,0.66,삼양사
18,삼양패키징,99.0,NaN,NaN,NaN


In [573]:
dfOwn.loc[18, 'own'] = 0.34
dfOwn.loc[18, 'own2'] = 0.34

In [574]:
dfOwn = dfOwn.drop(16)

In [575]:
grpname2 = '삼양'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

samyang


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,삼남석유화학,0.0,0.000000,0.000000,NaN,비금융회사,삼양,samyang,512
1,삼남석유화학,1.0,0.400000,0.400000,삼양홀딩스,비금융회사,삼양,samyang,512
2,삼남석유화학,99.0,0.600000,0.600000,NaN,비금융회사,삼양,samyang,512
3,삼양데이타시스템,0.0,0.000000,0.000000,NaN,비금융회사,삼양,samyang,512
4,삼양데이타시스템,1.0,1.000000,1.000000,삼양홀딩스,비금융회사,삼양,samyang,512
5,삼양데이타시스템,99.0,0.000000,0.000000,NaN,비금융회사,삼양,samyang,512
6,삼양사,0.0,0.014160,0.094156,NaN,비금융회사,삼양,samyang,512
7,삼양사,1.0,0.596925,0.596925,삼양홀딩스,비금융회사,삼양,samyang,512
8,삼양사,99.0,0.308919,0.308919,NaN,비금융회사,삼양,samyang,512
9,삼양에프앤비,0.0,0.000000,0.000000,NaN,비금융회사,삼양,samyang,512


# 대방건설

In [576]:
df = pd.read_excel('../data/ownership-status/raw/2021/대방건설.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [577]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [578]:
dfOwn

,firmname,type,own,own2,ownername
0,노블랜드,1.0,1.000000,1.000000,대방건설
1,대덕하우징씨스템,0.0,1.000000,1.000000,NaN
2,대방개발기업,1.0,1.000000,1.000000,대방건설
3,대방건설,0.0,1.000000,1.000000,NaN
4,대방건설동탄,1.0,0.950000,0.950000,대방건설
5,대방건설동탄,99.0,0.050000,0.050000,NaN
6,대방덕은,1.0,0.950000,0.950000,대방건설
7,대방덕은,99.0,0.050000,0.050000,NaN
8,대방디엠시티,0.0,NaN,0.033333,NaN
9,대방디엠시티,1.0,0.966667,0.966667,대방건설


In [579]:
grpname2 = '대방건설'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

dbcons


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,노블랜드,1.0,1.000000,1.000000,대방건설,비금융회사,대방건설,dbcons,10150
1,대덕하우징씨스템,0.0,1.000000,1.000000,NaN,비금융회사,대방건설,dbcons,10150
2,대방개발기업,1.0,1.000000,1.000000,대방건설,비금융회사,대방건설,dbcons,10150
3,대방건설,0.0,1.000000,1.000000,NaN,비금융회사,대방건설,dbcons,10150
4,대방건설동탄,1.0,0.950000,0.950000,대방건설,비금융회사,대방건설,dbcons,10150
5,대방건설동탄,99.0,0.050000,0.050000,NaN,비금융회사,대방건설,dbcons,10150
6,대방덕은,1.0,0.950000,0.950000,대방건설,비금융회사,대방건설,dbcons,10150
7,대방덕은,99.0,0.050000,0.050000,NaN,비금융회사,대방건설,dbcons,10150
8,대방디엠시티,0.0,NaN,0.033333,NaN,비금융회사,대방건설,dbcons,10150
9,대방디엠시티,1.0,0.966667,0.966667,대방건설,비금융회사,대방건설,dbcons,10150


# 현대해상화재보험

In [580]:
df = pd.read_excel('../data/ownership-status/raw/2021/현대해상.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [581]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
밀텍    1000.0
Name: own2, dtype: float64

In [582]:
df[df['소속회사명'] == '밀텍']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
409,비금융회사,밀텍,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
410,비금융회사,밀텍,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
411,비금융회사,밀텍,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
412,비금융회사,밀텍,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
413,비금융회사,밀텍,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
414,비금융회사,밀텍,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
415,비금융회사,밀텍,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
416,비금융회사,밀텍,동일인측,친족,인척4촌이내,이성태,50000,25.00,-,-,50000.0,25.00,NaN,NaN
417,비금융회사,밀텍,동일인측,친족,인척4촌이내,정희숙,24000,12.00,-,-,24000.0,12.00,NaN,NaN
418,비금융회사,밀텍,동일인측,친족,친족합계,/,74000,37.00,-,-,74000.0,37.00,0.0,0.0


In [583]:
df.loc[430, '합계주식수'] = 200000

In [584]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [585]:
grpname2 = '현대해상화재보험'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

HI


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,더패밀리랩,0.0,0.000000,0.333500,NaN,비금융회사,현대해상화재보험,HI,1402703
1,더패밀리랩,1.0,0.571306,0.571306,에이치지이니셔티브,비금융회사,현대해상화재보험,HI,1402703
3,더패밀리랩,99.0,0.095194,0.095194,NaN,비금융회사,현대해상화재보험,HI,1402703
4,마이금융파트너,0.0,0.000000,0.000000,NaN,금융회사,현대해상화재보험,HI,1402703
5,마이금융파트너,1.0,1.000000,1.000000,현대해상화재보험,금융회사,현대해상화재보험,HI,1402703
7,마이금융파트너,99.0,0.000000,0.000000,NaN,금융회사,현대해상화재보험,HI,1402703
8,밀텍,0.0,0.370000,0.863000,NaN,비금융회사,현대해상화재보험,HI,1402703
11,밀텍,99.0,0.137000,0.137000,NaN,비금융회사,현대해상화재보험,HI,1402703
12,솔로몬엠,0.0,0.000000,0.000000,NaN,비금융회사,현대해상화재보험,HI,1402703
13,솔로몬엠,1.0,1.000000,1.000000,솔로몬테크노서플라이,비금융회사,현대해상화재보험,HI,1402703


# 엠디엠

In [586]:
df = pd.read_excel('../data/ownership-status/raw/2021/엠디엠.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [587]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
비오에스마케팅     2.0
알에스에스테크윈    2.0
엠앤케이        2.0
Name: own2, dtype: float64

In [588]:
df[df['소속회사명'] == '비오에스마케팅']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
202,비금융회사,비오에스마케팅,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
203,비금융회사,비오에스마케팅,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
204,비금융회사,비오에스마케팅,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
205,비금융회사,비오에스마케팅,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
206,비금융회사,비오에스마케팅,동일인측,친족,인척4촌이내,김영,10000,100%,-,-,10000.0,100%,NaN,NaN
207,비금융회사,비오에스마케팅,동일인측,친족,친족합계,-,10000,100%,-,-,10000.0,100%,0.0,0.0
208,비금융회사,비오에스마케팅,동일인측,비영리법인,비영리법인,-,-,-,-,-,NaN,-,NaN,0.0
209,비금융회사,비오에스마케팅,동일인측,등기된임원,등기된임원,-,-,-,-,-,NaN,-,NaN,0.0
210,비금융회사,비오에스마케팅,동일인측,자기주식,자기주식,-,-,-,-,-,NaN,-,NaN,0.0
211,비금융회사,비오에스마케팅,동일인측,계열회사(국내+해외),계열회사(국내+해외),-,-,-,-,-,NaN,-,1.0,1.0


In [589]:
df.loc[216, '동일인과의관계'] = '총계'

In [590]:
df[df['소속회사명'] == '알에스에스테크윈']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
233,비금융회사,알에스에스테크윈,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
234,비금융회사,알에스에스테크윈,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
235,비금융회사,알에스에스테크윈,동일인측,친족,혈족2~4촌,문양현,103400,51.70%,-,-,103400.0,51.70%,NaN,NaN
236,비금융회사,알에스에스테크윈,동일인측,친족,혈족5~6촌,문동석,12120,6.06%,-,-,12120.0,6.06%,NaN,NaN
237,비금융회사,알에스에스테크윈,동일인측,친족,인척4촌이내,이월순,15000,7.50%,-,-,15000.0,7.50%,NaN,NaN
238,비금융회사,알에스에스테크윈,동일인측,친족,친족합계,-,130520,65.26%,-,-,130520.0,65.26%,0.0,0.0
239,비금융회사,알에스에스테크윈,동일인측,비영리법인,비영리법인,-,-,-,-,-,NaN,-,NaN,0.0
240,비금융회사,알에스에스테크윈,동일인측,등기된임원,등기된임원,-,-,-,-,-,NaN,-,NaN,0.0
241,비금융회사,알에스에스테크윈,동일인측,자기주식,자기주식,-,-,-,-,-,NaN,-,NaN,0.0
242,비금융회사,알에스에스테크윈,동일인측,계열회사(국내+해외),계열회사(국내+해외),-,-,-,-,-,NaN,-,1.0,1.0


In [591]:
df.loc[247, '동일인과의관계'] = '총계'

In [592]:
df[df['소속회사명'] == '엠앤케이']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
217,비금융회사,엠앤케이,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
218,비금융회사,엠앤케이,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
219,비금융회사,엠앤케이,동일인측,친족,혈족2~4촌,문순애,24000,40.00%,-,-,24000.0,40.00%,NaN,NaN
220,비금융회사,엠앤케이,동일인측,친족,혈족2~4촌,권미희,12000,20.00%,-,-,12000.0,20.00%,NaN,NaN
221,비금융회사,엠앤케이,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
222,비금융회사,엠앤케이,동일인측,친족,인척4촌이내,김영,6000,10.00%,-,-,6000.0,10.00%,NaN,NaN
223,비금융회사,엠앤케이,동일인측,친족,친족합계,-,42000,70.00%,-,-,42000.0,70.00%,0.0,0.0
224,비금융회사,엠앤케이,동일인측,비영리법인,비영리법인,-,-,-,-,-,NaN,-,NaN,0.0
225,비금융회사,엠앤케이,동일인측,등기된임원,등기된임원,-,-,-,-,-,NaN,-,NaN,0.0
226,비금융회사,엠앤케이,동일인측,자기주식,자기주식,-,-,-,-,-,NaN,-,NaN,0.0


In [593]:
df.loc[232, '동일인과의관계'] = '총계'

In [594]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [595]:
grpname2 = '엠디엠'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

MDM


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,더엠리테일,0.0,1.000000,1.000000,NaN,비금융회사,엠디엠,MDM,3801304
2,더엠리테일,99.0,0.000000,0.000000,NaN,비금융회사,엠디엠,MDM,3801304
3,비오에스마케팅,0.0,1.000000,1.000000,NaN,비금융회사,엠디엠,MDM,3801304
5,비오에스마케팅,99.0,0.000000,0.000000,NaN,비금융회사,엠디엠,MDM,3801304
6,쏘울컬렉션,0.0,1.000000,1.000000,NaN,비금융회사,엠디엠,MDM,3801304
8,쏘울컬렉션,99.0,0.000000,0.000000,NaN,비금융회사,엠디엠,MDM,3801304
9,알에스에스테크윈,0.0,0.652600,0.652600,NaN,비금융회사,엠디엠,MDM,3801304
11,알에스에스테크윈,99.0,0.347400,0.347400,NaN,비금융회사,엠디엠,MDM,3801304
12,에스비씨피에프브이,0.0,0.000000,0.000000,NaN,비금융회사,엠디엠,MDM,3801304
13,에스비씨피에프브이,1.0,0.500000,0.500000,엠디엠플러스,비금융회사,엠디엠,MDM,3801304


# 아이에스지주

In [596]:
df = pd.read_excel('../data/ownership-status/raw/2021/아이에스지주.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [597]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

Series([], Name: own2, dtype: float64)

In [598]:
dfOwn

,firmname,type,own,own2,ownername
0,골든에코,0.0,0.00,0.00,NaN
1,골든에코,1.0,0.55,0.55,인선이엔티
2,골든에코,1.0,NaN,NaN,-
3,골든에코,99.0,0.45,0.45,NaN
4,나래이앤씨,0.0,0.00,0.00,NaN
...,...,...,...,...,...
179,파주비앤알,99.0,0.00,0.00,NaN
180,해니건설,0.0,0.00,0.00,NaN
181,해니건설,1.0,1.00,1.00,아이에스동서
182,해니건설,1.0,NaN,NaN,-


In [599]:
grpname2 = '아이에스지주'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

IS


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,골든에코,0.0,0.00,0.00,NaN,비금융회사,아이에스지주,IS,3501209
1,골든에코,1.0,0.55,0.55,인선이엔티,비금융회사,아이에스지주,IS,3501209
3,골든에코,99.0,0.45,0.45,NaN,비금융회사,아이에스지주,IS,3501209
4,나래이앤씨,0.0,0.00,0.00,NaN,비금융회사,아이에스지주,IS,3501209
5,나래이앤씨,1.0,1.00,1.00,아이에스동서,비금융회사,아이에스지주,IS,3501209
...,...,...,...,...,...,...,...,...,...
177,파주비앤알,1.0,1.00,1.00,인선이엔티,비금융회사,아이에스지주,IS,3501209
179,파주비앤알,99.0,0.00,0.00,NaN,비금융회사,아이에스지주,IS,3501209
180,해니건설,0.0,0.00,0.00,NaN,비금융회사,아이에스지주,IS,3501209
181,해니건설,1.0,1.00,1.00,아이에스동서,비금융회사,아이에스지주,IS,3501209


# 중앙

In [600]:
df = pd.read_excel('../data/ownership-status/raw/2021/중앙홀딩스.xlsx')
df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
df['합계주식수'] = df['합계주식수'].copy().astype(float)

In [601]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
더기프팅컴퍼니               1.430740
베이스에이스트링창업벤처전문사모투자    0.984085
베이스인베스트먼트             1.000000
Name: own2, dtype: float64

In [602]:
df[df['소속회사명'] == '더기프팅컴퍼니']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
88,비금융회사,더기프팅컴퍼니,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
89,비금융회사,더기프팅컴퍼니,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
90,비금융회사,더기프팅컴퍼니,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
91,비금융회사,더기프팅컴퍼니,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
92,비금융회사,더기프팅컴퍼니,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
93,비금융회사,더기프팅컴퍼니,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
94,비금융회사,더기프팅컴퍼니,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
95,비금융회사,더기프팅컴퍼니,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
96,비금융회사,더기프팅컴퍼니,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
97,비금융회사,더기프팅컴퍼니,동일인측,친족,친족합계,/,-,-,-,-,NaN,-,0.0,0.0


In [603]:
df.loc[109, '합계주식수'] = 38406.0 + 16543.0

In [604]:
df['ownCat'] = df.apply(catOwn, axis=1)
df['ownCat2'] = df.apply(catOwn2, axis=1)
dfShares = df[df['동일인과의관계2'] == '총계'][['소속회사명', '합계주식수']]
dfShares.columns = ['소속회사명', '주식수총계']
dfMrg = df.merge(dfShares, on='소속회사명')
dfMrg['shares'] = dfMrg['합계주식수'] / dfMrg['주식수총계']
dfOwn = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat'])['shares'].sum()).reset_index()
dfOwn = dfOwn[dfOwn['ownCat'] != 1]
dfOwn2 = pd.DataFrame(dfMrg.groupby(['소속회사명', 'ownCat2'])['shares'].sum()).reset_index()
dfOwn2 = dfOwn2[dfOwn2['ownCat2'] != 1]
dfOwn2.columns = ['소속회사명', 'ownCat', 'shares']
dfMrgd = dfOwn.merge(dfOwn2, on=['소속회사명', 'ownCat'], how='outer')
dfMrgd.columns = ['소속회사명', 'ownCat', 'own', 'own2']
dfMrgd['ownername'] = np.nan
dfOwn1 = dfMrg[dfMrg['ownCat'] == 1][['소속회사명', 'ownCat', 'shares', '성명']]
dfOwn1.columns = ['소속회사명', 'ownCat', 'own', 'ownername']
dfOwn1['own2'] = dfOwn1['own']
dfOwn1 = dfOwn1[['소속회사명', 'ownCat', 'own', 'own2', 'ownername']]
dfOwn1 = dfOwn1[~dfOwn1['ownername'].isna()]
dfOwn = pd.concat([dfMrgd, dfOwn1])
dfOwn = dfOwn.sort_values(['소속회사명', 'ownCat'])
dfOwn.columns = ['firmname', 'type', 'own', 'own2', 'ownername']
dfOwn = dfOwn.reset_index()
dfOwn = dfOwn.drop(columns=['index'])
dfOwn.groupby(['firmname'])['own2'].sum()[dfOwn.groupby(['firmname'])['own2'].sum() != 1]

firmname
베이스에이스트링창업벤처전문사모투자    0.984085
베이스인베스트먼트             1.000000
Name: own2, dtype: float64

In [605]:
df[df['소속회사명'] == '베이스에이스트링창업벤처전문사모투자']

,금융회사,소속회사명,동일인과의관계,동일인과의관계2,동일인과의관계3,성명,보통주주식수,보통주지분율,우선주주식수,우선주지분율,합계주식수,합계지분율,ownCat,ownCat2
1157,금융회사,베이스에이스트링창업벤처전문사모투자,동일인측,동일인,동일인,-,-,-,-,-,NaN,-,0.0,0.0
1158,금융회사,베이스에이스트링창업벤처전문사모투자,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
1159,금융회사,베이스에이스트링창업벤처전문사모투자,동일인측,친족,배우자/혈족1촌,-,-,-,-,-,NaN,-,NaN,NaN
1160,금융회사,베이스에이스트링창업벤처전문사모투자,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
1161,금융회사,베이스에이스트링창업벤처전문사모투자,동일인측,친족,혈족2~4촌,-,-,-,-,-,NaN,-,NaN,NaN
1162,금융회사,베이스에이스트링창업벤처전문사모투자,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
1163,금융회사,베이스에이스트링창업벤처전문사모투자,동일인측,친족,혈족5~6촌,-,-,-,-,-,NaN,-,NaN,NaN
1164,금융회사,베이스에이스트링창업벤처전문사모투자,동일인측,친족,인척4촌이내,신현성,45000,1.59%,-,-,45000.0,1.59%,NaN,NaN
1165,금융회사,베이스에이스트링창업벤처전문사모투자,동일인측,친족,인척4촌이내,-,-,-,-,-,NaN,-,NaN,NaN
1166,금융회사,베이스에이스트링창업벤처전문사모투자,동일인측,친족,친족합계,/,45000,1.59%,-,-,45000.0,1.59%,0.0,0.0


In [606]:
grpname2 = '중앙'
dfFinancial = df[['금융회사', '소속회사명']].drop_duplicates().reset_index().drop(columns=['index'])
dfFinancial.columns = ['금융비금융', 'firmname']
dfOwnInput = dfOwn.merge(dfFinancial, on='firmname', how='left')
dfOwnInput['grpname2'] = grpname2
dfOwnInput = dfOwnInput.merge(dfGroupNmId21, on='grpname2', how='left')
dfOwnInput = dfOwnInput[dfOwnInput.ownername != '-']
grpname = dfGroupNmId21[dfGroupNmId21['grpname2'] == grpname2]['grpname'].values[0]
print(grpname)
dfOwnInput.to_excel(f"../data/ownership-input/2021/{grpname}2021raw.xlsx", index=False)
dfOwnInput

joongang


,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,그린터틀,0.0,0.8,0.8,NaN,비금융회사,중앙,joongang,36006
3,그린터틀,99.0,0.2,0.2,NaN,비금융회사,중앙,joongang,36006
4,다보중앙,0.0,1.0,1.0,NaN,금융회사,중앙,joongang,36006
5,다보중앙,99.0,0.0,0.0,NaN,금융회사,중앙,joongang,36006
6,더기프팅컴퍼니,0.0,0.0,0.0,NaN,비금융회사,중앙,joongang,36006
...,...,...,...,...,...,...,...,...,...
222,휘닉스중앙제주,1.0,1.0,1.0,휘닉스중앙,비금융회사,중앙,joongang,36006
224,휘닉스중앙제주,99.0,0.0,0.0,NaN,비금융회사,중앙,joongang,36006
225,휘닉스평창,0.0,0.0,0.0,NaN,비금융회사,중앙,joongang,36006
226,휘닉스평창,1.0,1.0,1.0,휘닉스중앙,비금융회사,중앙,joongang,36006
